In [1]:
#%matplotlib inline # set backend option to inline
%matplotlib notebook 
# set backend option for interactive plotting

import matplotlib 
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

import os
import os.path as osp
import numpy as np
import pandas as pd
import json


In [2]:
from datetime import datetime , timedelta, date
import pytz
#print(datetime.utcfromtimestamp(1492043679))   #2017-04-13 00:34:39
#print(datetime.utcfromtimestamp(1492372131))   #2017-04-16 19:48:51
#print(datetime.strptime("170416-194851", "%y%m%d-%H%M%S"))   #2017-04-16 19:48:51

f_utc2ts_naive = lambda t: datetime.timestamp(datetime.strptime(t, '%Y-%m-%d %H:%M:%S'))
g_utc2ts_naive = lambda t: datetime.timestamp(datetime.strptime(t, '%b %d, %Y, %I:%M:%S %p'))

f_utc2dt_naive = lambda t: (datetime.strptime(t, '%Y-%m-%d %H:%M:%S'))
g_utc2dt_naive = lambda t: (datetime.strptime(t, '%b %d, %Y, %I:%M:%S %p'))


#time zone aware datetime 
Time_Zone_US_E = pytz.timezone('US/Eastern')
f_utc2ts_timezone = lambda t: datetime.timestamp(datetime.strptime(t, '%Y-%m-%d %H:%M:%S'))
g_utc2ts_timezone= lambda t: datetime.timestamp(datetime.strptime(t, '%b %d, %Y, %I:%M:%S %p'))

f_utc2dt_timezone = lambda t: (datetime.strptime(t, '%Y-%m-%d %H:%M:%S'))
g_utc2dt_timezone = lambda t: (datetime.strptime(t, '%b %d, %Y, %I:%M:%S %p'))

In [3]:
import pytz
pst = pytz.timezone('US/Eastern')
d = datetime.fromtimestamp(1.491404e+09)
print(d)
print(d.astimezone(pst))
d1 = datetime.utcfromtimestamp(1.491404e+09)
print(d1)
print(d1.astimezone(pst))
timezone_startTime=(datetime.utcfromtimestamp(float(1.491404e+09))).astimezone(pst)
print(timezone_startTime)

2017-04-05 19:53:20
2017-04-05 10:53:20-04:00
2017-04-05 14:53:20
2017-04-05 05:53:20-04:00
2017-04-05 05:53:20-04:00


In [4]:
dpd_dir=os.getcwd()
dpd_n='subject_diary_data-migraine-app-6a5b55f9-423f-463a-a345-3b268a63a8e9-export.json'
#print(dpd_dir)

dpd_path=os.path.join(dpd_dir, dpd_n)
#print("Does the file path exist?: ",os.path.exists(dpd_path))
#print("Do I have read access to the file?: ",os.access(dpd_path, os.R_OK))

#Read json file using pd
dpd=pd.read_json(dpd_path)
#print(dpd.shape)

# The timestamps are labeled as columns. Change to rows
dpd=dpd.transpose()
# Convert row indices, which are time stamps, to a dataframe column
dpd.reset_index(inplace = True)
#print(dpd.shape, dpd.head())dpd['ts_diary'] = dpd['dt_diary'].map(datetime.timestamp)

###################################################################################
# Change column name 'index' to 'dt_diary'
#....................................................................................
#print(dpd.index)   # RangeIndex(start=0, stop=92, step=1)
#print(dpd[['index']].dtypes)  #index    datetime64[ns]  #dtype: object
#print(dpd[['index']].columns[0])   #index
#print(dpd.columns)  # Index(['index', 'HADMIGRAINE', 'HELPMIGRAINETODAY', 'LURKINGMIGRAINE','MIGRAINEEND', 'MIGRAINESEVERITY', 'MIGRAINESTART','SLEEPDURATIONHOURS', 'SLEEPDURATIONMINUTES', 'SLEEPQUALITY','STRESSLEVEL', 'SYMPTOMSTODAY', 'TRIGGERSTODAY'],dtype='object')
#....................................................................................
x = list(dpd.columns); #print(x)  # ['index', 'HADMIGRAINE', 'HELPMIGRAINETODAY', 'LURKINGMIGRAINE', 'MIGRAINEEND', 'MIGRAINESEVERITY', 'MIGRAINESTART', 'SLEEPDURATIONHOURS', 'SLEEPDURATIONMINUTES', 'SLEEPQUALITY', 'STRESSLEVEL', 'SYMPTOMSTODAY', 'TRIGGERSTODAY']
x[0] = 'dt_diary' ; #print(x) # ['dt_diary', 'HADMIGRAINE', 'HELPMIGRAINETODAY', 'LURKINGMIGRAINE', 'MIGRAINEEND', 'MIGRAINESEVERITY', 'MIGRAINESTART', 'SLEEPDURATIONHOURS', 'SLEEPDURATIONMINUTES', 'SLEEPQUALITY', 'STRESSLEVEL', 'SYMPTOMSTODAY', 'TRIGGERSTODAY']
dpd.columns = x ; #print(dpd.columns) # Index(['dt_diary', 'HADMIGRAINE', 'HELPMIGRAINETODAY', 'LURKINGMIGRAINE', 'MIGRAINEEND', 'MIGRAINESEVERITY', 'MIGRAINESTART', 'SLEEPDURATIONHOURS', 'SLEEPDURATIONMINUTES', 'SLEEPQUALITY', 'STRESSLEVEL', 'SYMPTOMSTODAY', 'TRIGGERSTODAY'], dtype='object')
print(dpd.head(2))


             dt_diary HADMIGRAINE HELPMIGRAINETODAY LURKINGMIGRAINE  \
0 2017-04-21 19:23:19       False               NaN              No   
1 2017-04-21 20:45:56       False     [Meditation ]              No   

  MIGRAINEEND MIGRAINESEVERITY MIGRAINESTART SLEEPDURATIONHOURS  \
0         NaN              NaN           NaN                  7   
1         NaN              NaN           NaN                  7   

  SLEEPDURATIONMINUTES SLEEPQUALITY STRESSLEVEL  \
0                   15            3           2   
1                                 3           2   

                     SYMPTOMSTODAY                         TRIGGERSTODAY  
0                              NaN                                   NaN  
1  [Dizziness, Fatigue, Body Pain]  [Drop in Barometric Pressure, Smoke]  


C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [5]:
# Diary entries are not in chronological order. So sort
dpd.sort_values(by = 'dt_diary', inplace = True)
# The row index/labels are maintained, so reset
dpd.reset_index(inplace = True)
#print(dpd['index'])
# Finally, drop the column with the old row indices/labels
dpd.drop(['index'], axis = 1, inplace = True)
#dpd[12:30]

In [6]:
r=lambda ts: pytz.UTC.localize(ts)
# # pytz.UTC.localize(d)
dpd['dt_diary_tz']=dpd['dt_diary'].map(r)

In [7]:
dpd['MIGRAINESTART_TimeZone'] = (dpd['MIGRAINESTART'].loc[ dpd['MIGRAINESTART'].notnull() ]).map(g_utc2dt_timezone)
dpd['MIGRAINEEND_TimeZone'] = (dpd['MIGRAINEEND'].loc[ dpd['MIGRAINEEND'].notnull() ]).map(g_utc2dt_timezone)

In [8]:
dpd['MIGRAINESTART_TimeZone'].loc[ dpd['MIGRAINESTART_TimeZone'].notnull()]

10   2017-04-28 09:30:14
19   2017-05-09 13:30:57
23   2017-05-13 13:30:08
32   2017-05-23 17:00:06
34   2017-05-25 17:30:09
38   2017-05-30 11:30:42
39   2017-05-31 10:30:14
43   2017-06-04 06:00:42
55   2017-06-16 06:00:07
57   2017-06-18 02:00:51
71   2017-07-04 10:00:15
75   2017-07-08 14:31:27
76   2017-07-09 03:00:40
81   2017-07-14 16:30:58
83   2017-07-16 10:31:34
Name: MIGRAINESTART_TimeZone, dtype: datetime64[ns]

In [9]:
condition_dpd11 = ( (dpd['MIGRAINESTART_TimeZone'].isnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )
# dpd['MIGRAINESTART_TimeZone']=dpd[condition_dpd11].MIGRAINEEND_TimeZone-timedelta(hours = 15)


In [10]:
condition_dpd11 = ( (dpd['MIGRAINESTART'].isnull() ) ) & ( dpd['MIGRAINEEND'].notnull() )


In [11]:
datey=[]
for i,row in dpd.loc[condition_dpd11, :].iterrows() : 
    r=(row['MIGRAINEEND_TimeZone']-timedelta(hours = 15))
    datey.append(r)
    print(r)
    dpd.at[i, 'Extrappolated']=1
    dpd.at[i, 'MIGRAINESTART_TimeZone'] = r
#     row['MIGRAINESTART_TimeZone']=r

2017-04-22 08:14:59
2017-04-23 02:40:46
2017-04-25 07:15:20
2017-05-01 06:34:39
2017-05-15 04:15:22
2017-05-18 06:00:47
2017-05-21 08:22:04
2017-05-27 07:43:14
2017-06-02 06:30:03
2017-06-04 23:30:01
2017-06-07 06:51:01
2017-06-08 07:47:55
2017-06-09 07:46:18
2017-06-15 07:30:07
2017-06-17 00:30:07
2017-06-20 05:46:15
2017-06-21 05:00:54
2017-06-24 07:05:47
2017-06-26 06:27:00
2017-06-27 02:45:36
2017-06-28 07:45:48
2017-06-30 00:00:46
2017-07-02 01:00:11
2017-07-20 21:00:38
2017-07-22 01:45:31


In [12]:
# Time Stamp
dpd['ts_MSTART'] = None
dpd['ts_MEND'] =  None

# Convert the string to a unix time stamp
dpd['ts_MSTART'] = (dpd['MIGRAINESTART'].loc[ dpd['MIGRAINESTART'].notnull() ]).map(g_utc2ts_naive)
dpd['ts_MEND'] = (dpd['MIGRAINEEND'].loc[ dpd['MIGRAINEEND'].notnull() ]).map(g_utc2ts_naive)

# Date Time
dpd['dt_MSTART'] = None
dpd['dt_MEND'] =  None

# Convert the string to a datetime object
dpd['dt_MSTART'] = (dpd['MIGRAINESTART'].loc[ dpd['MIGRAINESTART'].notnull() ]).map(g_utc2dt_timezone)
dpd['dt_MEND'] = (dpd['MIGRAINEEND'].loc[ dpd['MIGRAINEEND'].notnull() ]).map(g_utc2dt_timezone)

# dpd['dt_diary'] = (dpd['MIGRAINESTART'].loc[ dpd['MIGRAINESTART'].notnull() ]).map(g_utc2dt_timezone)

# Time delta
dpd["ts_delta"] = None
dpd["dt_delta"] = None

dpd["ts_delta"] = dpd['ts_MEND'] - dpd['ts_MSTART']
dpd["dt_delta"] = dpd['dt_MEND'] - dpd['dt_MSTART']

In [13]:
# dpd['dt_MSTART']

In [14]:
dpd['dt_delta'].mean()

Timedelta('0 days 15:06:47.666666')

In [15]:
# Add migraine status 'mig_status': 4 = migraine with start and end times provided;
#                                   3 = migraine with only end time
#                                   2 = migraine with only start time
#                                   1 = migraine with no start or end time
#                                   0 = no migraine
condition_dpd4 = ( (dpd['MIGRAINESTART'].notnull() ) ) & ( dpd['MIGRAINEEND'].notnull() )
condition_dpd3 = (dpd['MIGRAINESTART'].isnull() )  & ( dpd['MIGRAINEEND'].notnull())
condition_dpd2 = (dpd['MIGRAINESTART'].notnull() )  & ( dpd['MIGRAINEEND'].isnull())
condition_dpd1 =  (dpd['HADMIGRAINE'] == True) & (dpd['MIGRAINESTART'].isnull() )  & ( dpd['MIGRAINEEND'].isnull())
condition_dpd0 =  (dpd['HADMIGRAINE'] == False)  # Probably redundant to add 

dpd['mig_status']=(4 * condition_dpd4) + (3 * condition_dpd3) + (2 * condition_dpd2) + (1 * condition_dpd1) + (0 * condition_dpd0)

In [16]:
plt.ion()
fig_dpdX = plt.figure(figsize = (9,3))
sfig_dpdX1 = fig_dpdX.add_subplot(111)
sfig_dpdX1.set_xlim(xmin = -5, xmax = dpd.shape[0]+4)
sfig_dpdX1.set_ylim(ymin = dpd['dt_diary'].min() - timedelta(days = 5)  , ymax= dpd['dt_diary'].max() + timedelta(days = 5)  )
sfig_dpdX1.plot(dpd.loc[condition_dpd4,'dt_diary'], 'ro')
sfig_dpdX1.plot(dpd.loc[condition_dpd3,'dt_diary'], 'go')
sfig_dpdX1.plot(dpd.loc[condition_dpd2,'dt_diary'], 'g*')
sfig_dpdX1.plot(dpd.loc[condition_dpd1,'dt_diary'], 'g^')
sfig_dpdX1.plot(dpd.loc[condition_dpd0,'dt_diary'], 'bo')

for i, row in dpd.loc[condition_dpd4, :].iterrows() :
    #sfig_dpd01_2.axvline(row['dt_diary'], color = 'green', linewidth = 2)
    #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
    
    sfig_dpdX1.axhspan(row['MIGRAINESTART'],row['MIGRAINEEND'], color = 'orchid', alpha = 0.5)
    sfig_dpdX1.axhline(row['MIGRAINESTART'], color = 'orchid', linewidth = 2, linestyle = '--')
    sfig_dpdX1.axhline(row['MIGRAINEEND'], color = 'orchid', linewidth = 2, linestyle = ':')
    sfig_dpdX1.axhline(row['dt_diary'], color = 'orchid', linewidth = 1, linestyle = '-')


for i, row in dpd.loc[condition_dpd3, :].iterrows() :
    sfig_dpdX1.axhline(row['MIGRAINEEND'], color = 'green', linewidth = 2, linestyle = ':')
    sfig_dpdX1.axhline(row['dt_diary'], color = 'green', linewidth = 1, linestyle = '-', alpha = 0.7)

for i, row in dpd.loc[condition_dpd1, :].iterrows() :
    sfig_dpdX1.axhline(row['MIGRAINEEND'], color = 'blue', linewidth = 2, linestyle = ':')
    sfig_dpdX1.axhline(row['dt_diary'], color = 'blue', linewidth = 1, linestyle = '-', alpha = 0.7)

# index 74 - 87
# index 69 - 72
# index 49 - 53
# index 42 - 44
# index 36 - 40
# index 30 - 35
# index 12 - 24
# index 10 - 11

<IPython.core.display.Javascript object>

C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\pandas\plotting\_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [17]:
#dpd.mig_status.cumsum() == 0

In [18]:
#x  = dpd['SLEEPDURATIONHOURS'];y = dpd['SLEEPDURATIONMINUTES']

In [19]:
#isinstance('', str)
#np.isnan(np.nan)

#x.loc[x.notnull()].map(format_sleep)
#y.loc[y.notnull()].map(lambda x : (float("0" + str(x)))).map(lambda x : timedelta(minutes = x ))

In [20]:
x  = dpd['SLEEPDURATIONHOURS'].copy() ;y = dpd['SLEEPDURATIONMINUTES'].copy()
dpd['dt_deltasleep_hrs'] =None
dpd['dt_deltasleep_mins'] = None

dpd['dt_deltasleep_hrs'] = x.loc[x.notnull()].map(lambda x : (float("0" + str(x)))).map(lambda x : timedelta(hours = x ))
dpd['dt_deltasleep_mins'] = y.loc[y.notnull()].map(lambda x : (float("0" + str(x)))).map(lambda x : timedelta(minutes = x ))

dpd['dt_sleep'] = dpd['dt_deltasleep_hrs'].copy()

cond1 = dpd['dt_deltasleep_hrs'].notnull() & dpd['dt_deltasleep_mins'].isnull()
if cond1.any() :
    dpd['dt_sleep'].loc[cond1] = dpd['dt_deltasleep_hrs'].loc[cond1]

cond2 = dpd['dt_deltasleep_hrs'].notnull() & dpd['dt_deltasleep_mins'].notnull()
if cond2.any():
    dpd['dt_sleep'].loc[cond2] = dpd['dt_deltasleep_hrs'].loc[cond2] + dpd['dt_deltasleep_mins'].loc[cond2] 

cond3 = dpd['dt_deltasleep_hrs'].isnull() & dpd['dt_deltasleep_mins'].notnull()
if cond3.any():
    dpd['dt_sleep'].loc[cond3] =  dpd['dt_deltasleep_mins'].loc[cond3] 
 
dpd['dt_sleepQ']=dpd['dt_sleep'].copy()
cond4 = dpd['dt_sleep'].notnull() & dpd['SLEEPQUALITY'].notnull()
if cond4.any() :
    dpd['dt_sleepQ'].loc[cond4] = dpd['dt_sleep'].loc[cond4] * (dpd['SLEEPQUALITY'].loc[cond4]).map(int)

    
del x, y

C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [21]:
# pt diary characteristics of sleep:
(dpd.SLEEPDURATIONMINUTES.notnull() & dpd.SLEEPDURATIONHOURS.isnull()).sum() #0
(dpd.SLEEPDURATIONMINUTES.isnull() & dpd.SLEEPDURATIONHOURS.isnull()).sum() #2
(dpd.SLEEPDURATIONMINUTES.isnull() & dpd.SLEEPDURATIONHOURS.notnull()).sum() #62
(dpd.SLEEPDURATIONMINUTES.notnull() & dpd.SLEEPDURATIONHOURS.notnull()).sum() #28

(dpd.SLEEPDURATIONMINUTES.notnull() & dpd.SLEEPDURATIONHOURS.isnull()& dpd.HADMIGRAINE == True).sum() # 0
(dpd.SLEEPDURATIONMINUTES.isnull() & dpd.SLEEPDURATIONHOURS.isnull()& dpd.HADMIGRAINE == True).sum() # 2
(dpd.SLEEPDURATIONMINUTES.isnull() & dpd.SLEEPDURATIONHOURS.notnull()& dpd.HADMIGRAINE == True).sum() # 28
(dpd.SLEEPDURATIONMINUTES.notnull() & dpd.SLEEPDURATIONHOURS.notnull()& dpd.HADMIGRAINE == True).sum() # 12

(dpd.SLEEPQUALITY.isnull()).sum() #3
(dpd.SLEEPQUALITY.isnull() & dpd.HADMIGRAINE == True).sum() #2
(dpd.SLEEPDURATIONMINUTES.isnull() & dpd.SLEEPDURATIONHOURS.isnull() & dpd.SLEEPQUALITY.isnull() & dpd.HADMIGRAINE == True).sum() #2

2

In [22]:
#dpd['SLEEPDURATIONHOURS'].loc[dpd['SLEEPDURATIONHOURS'].map(format_sleep)].map(lambda x: timedelta(hours = format_sleep(x)))

In [23]:
 #dpd['SLEEPQUALITY'].loc[cond4].map(int) *dpd['dt_sleep'].loc[cond4] 

In [24]:
#dpd['dt_deltasleep_mins'].notnull()
#dpd['dt_deltasleep_hrs'].loc[dpd['dt_deltasleep_mins'].notnull()]
#pd.concat([dpd[['dt_sleep','dt_deltasleep_hrs','dt_deltasleep_mins']],cond1, cond2, cond3], axis = 1)
#dpd[['SLEEPQUALITY', 'dt_sleepQ']]

In [25]:
#dpd.loc[0,['SLEEPQUALITY', 'dt_sleepQ', 'dt_sleep', 'dt_deltasleep_mins']].apply(type)
#timedelta(1)*2

In [26]:
#len()
#dpd['SLEEPDURATIONMINUTES'].loc[dpd['SLEEPDURATIONMINUTES'].notnull()].map(type) #> 0 #.map(lambda x: timedelta(minutes = float(x))

#( dpd['SLEEPDURATIONHOURS'].loc[dpd['SLEEPDURATIONHOURS'].notnull()] ).map(type)

In [27]:
#condition_dpdsleep = (dpd['SLEEPDURATIONHOURS'].notnull() & dpd['SLEEPDURATIONMINUTES'].notnull())
#dpd['dt_deltasleep'] = dpd['dt_deltasleep_hrs']
#dpd['dt_deltasleep'] = dpd.loc[condition_dpdsleep,'dt_deltasleep_hrs'] + dpd.loc[condition_dpdsleep,'dt_deltasleep_hrs']
#print(dpd.shape,'\n',dpd.columns)
#print(dpd.dtypes)

In [28]:
condition_dpd = ( (dpd['MIGRAINESTART'].notnull() ) ) & ( dpd['MIGRAINEEND'].notnull() )

dpd.loc[condition_dpd, ['MIGRAINESEVERITY', 'dt_delta']].shape #
#dpd['MIGRAINESEVERITY'].max() # 2

(15, 2)

In [29]:
#
MDUR_mean = ( dpd.loc[(condition_dpd), ['ts_delta', 'dt_delta']] ).mean()

print(MDUR_mean)

ts_delta                   54407.7
dt_delta    0 days 15:06:47.666666
dtype: object


In [30]:
# x=dpf.loc[date]
#     if (x['MIGRAINESTART'].notnull() ) ) & ( x['MIGRAINEEND'].notnull())):
#         sfig_dpd01_2.axvline(x['dt_diary'], color = 'green', linewidth = 2)
#         #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

#         sfig_dpd01_2.axvspan(x['MIGRAINESTART'],x['MIGRAINEEND'], color = 'orchid', alpha = 0.5)
#         sfig_dpd01_2.axvline(x['MIGRAINESTART'], color = 'red', linewidth = 1)
#         sfig_dpd01_2.axvline(x['MIGRAINEEND'], color = 'blue', linewidth = 1)

In [31]:
# # dpf.loc['2017-04-21'].empty
# strdate in dpf.index
# dpf.loc[date]

In [32]:
# print(x['MIGRAINESTART'])

In [33]:
dpd.MIGRAINESTART.head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: MIGRAINESTART, dtype: object

In [34]:
condition_dpd = ( (dpd['MIGRAINESTART_TimeZone'].notnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )
                  

In [35]:
condition_dpd11 = ( (dpd['MIGRAINESTART_TimeZone'].isnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )

In [36]:
dpd.loc[condition_dpd, :].count()

dt_diary                  40
HADMIGRAINE               40
HELPMIGRAINETODAY         37
LURKINGMIGRAINE            0
MIGRAINEEND               40
MIGRAINESEVERITY          40
MIGRAINESTART             15
SLEEPDURATIONHOURS        40
SLEEPDURATIONMINUTES      12
SLEEPQUALITY              40
STRESSLEVEL               40
SYMPTOMSTODAY             40
TRIGGERSTODAY             39
dt_diary_tz               40
MIGRAINESTART_TimeZone    40
MIGRAINEEND_TimeZone      40
Extrappolated             25
ts_MSTART                 15
ts_MEND                   40
dt_MSTART                 15
dt_MEND                   40
ts_delta                  15
dt_delta                  15
mig_status                40
dt_deltasleep_hrs         40
dt_deltasleep_mins        12
dt_sleep                  40
dt_sleepQ                 40
dtype: int64

In [37]:
dpd[( (dpd['MIGRAINESTART'].isnull() ) ) & ( dpd['MIGRAINEEND'].notnull() )].count()

dt_diary                  25
HADMIGRAINE               25
HELPMIGRAINETODAY         22
LURKINGMIGRAINE            0
MIGRAINEEND               25
MIGRAINESEVERITY          25
MIGRAINESTART              0
SLEEPDURATIONHOURS        25
SLEEPDURATIONMINUTES       6
SLEEPQUALITY              25
STRESSLEVEL               25
SYMPTOMSTODAY             25
TRIGGERSTODAY             24
dt_diary_tz               25
MIGRAINESTART_TimeZone    25
MIGRAINEEND_TimeZone      25
Extrappolated             25
ts_MSTART                  0
ts_MEND                   25
dt_MSTART                  0
dt_MEND                   25
ts_delta                   0
dt_delta                   0
mig_status                25
dt_deltasleep_hrs         25
dt_deltasleep_mins         6
dt_sleep                  25
dt_sleepQ                 25
dtype: int64

In [38]:
len(datey)

25

In [39]:
date_extra = [d.strftime('%Y-%m-%d') for d in datey]

In [40]:
date_extra

['2017-04-22',
 '2017-04-23',
 '2017-04-25',
 '2017-05-01',
 '2017-05-15',
 '2017-05-18',
 '2017-05-21',
 '2017-05-27',
 '2017-06-02',
 '2017-06-04',
 '2017-06-07',
 '2017-06-08',
 '2017-06-09',
 '2017-06-15',
 '2017-06-17',
 '2017-06-20',
 '2017-06-21',
 '2017-06-24',
 '2017-06-26',
 '2017-06-27',
 '2017-06-28',
 '2017-06-30',
 '2017-07-02',
 '2017-07-20',
 '2017-07-22']

In [41]:
dpd[(dpd['MIGRAINESTART'].isnull() & dpd['MIGRAINEEND'].notnull())].count()

dt_diary                  25
HADMIGRAINE               25
HELPMIGRAINETODAY         22
LURKINGMIGRAINE            0
MIGRAINEEND               25
MIGRAINESEVERITY          25
MIGRAINESTART              0
SLEEPDURATIONHOURS        25
SLEEPDURATIONMINUTES       6
SLEEPQUALITY              25
STRESSLEVEL               25
SYMPTOMSTODAY             25
TRIGGERSTODAY             24
dt_diary_tz               25
MIGRAINESTART_TimeZone    25
MIGRAINEEND_TimeZone      25
Extrappolated             25
ts_MSTART                  0
ts_MEND                   25
dt_MSTART                  0
dt_MEND                   25
ts_delta                   0
dt_delta                   0
mig_status                25
dt_deltasleep_hrs         25
dt_deltasleep_mins         6
dt_sleep                  25
dt_sleepQ                 25
dtype: int64

In [42]:
dpd[( (dpd['MIGRAINESTART'].isnull() ) ) & ( dpd['MIGRAINEEND'].notnull() )]

,dt_diary,HADMIGRAINE,HELPMIGRAINETODAY,LURKINGMIGRAINE,MIGRAINEEND,MIGRAINESEVERITY,MIGRAINESTART,SLEEPDURATIONHOURS,SLEEPDURATIONMINUTES,SLEEPQUALITY,...,ts_MEND,dt_MSTART,dt_MEND,ts_delta,dt_delta,mig_status,dt_deltasleep_hrs,dt_deltasleep_mins,dt_sleep,dt_sleepQ
3,2017-04-22 23:15:33,True,NaN,NaN,"Apr 22, 2017, 11:14:59 PM",1,NaN,6,30,3,...,1.492885e+09,NaT,2017-04-22 23:14:59,NaN,NaT,3,06:00:00,00:30:00,06:30:00,0 days 19:30:00
4,2017-04-23 21:23:50,True,[Sleep],NaN,"Apr 23, 2017, 5:40:46 PM",2,NaN,7,45,4,...,1.492951e+09,NaT,2017-04-23 17:40:46,NaN,NaT,3,07:00:00,00:45:00,07:45:00,1 days 07:00:00
6,2017-04-25 22:15:56,True,[Meditation ],NaN,"Apr 25, 2017, 10:15:20 PM",1,NaN,6,45,4,...,1.493141e+09,NaT,2017-04-25 22:15:20,NaN,NaT,3,06:00:00,00:45:00,06:45:00,1 days 03:00:00
12,2017-05-01 21:35:23,True,[Meditation ],NaN,"May 1, 2017, 9:34:39 PM",2,NaN,6,NaN,4,...,1.493656e+09,NaT,2017-05-01 21:34:39,NaN,NaT,3,06:00:00,NaT,06:00:00,1 days 00:00:00
25,2017-05-15 23:54:08,True,[Meditation ],NaN,"May 15, 2017, 7:15:22 PM",1,NaN,7,NaN,5,...,1.494858e+09,NaT,2017-05-15 19:15:22,NaN,NaT,3,07:00:00,NaT,07:00:00,1 days 11:00:00
28,2017-05-18 21:53:30,True,NaN,NaN,"May 18, 2017, 9:00:47 PM",1,NaN,7,30,4,...,1.495123e+09,NaT,2017-05-18 21:00:47,NaN,NaT,3,07:00:00,00:30:00,07:30:00,1 days 06:00:00
30,2017-05-21 23:22:42,True,NaN,NaN,"May 21, 2017, 11:22:04 PM",2,NaN,8,NaN,4,...,1.495391e+09,NaT,2017-05-21 23:22:04,NaN,NaT,3,08:00:00,NaT,08:00:00,1 days 08:00:00
36,2017-05-27 22:43:55,True,[Meditation ],NaN,"May 27, 2017, 10:43:14 PM",1,NaN,7,NaN,3,...,1.495907e+09,NaT,2017-05-27 22:43:14,NaN,NaT,3,07:00:00,NaT,07:00:00,0 days 21:00:00
41,2017-06-02 23:33:27,True,[Meditation ],NaN,"Jun 2, 2017, 9:30:03 PM",1,NaN,7,NaN,5,...,1.496421e+09,NaT,2017-06-02 21:30:03,NaN,NaT,3,07:00:00,NaT,07:00:00,1 days 11:00:00
44,2017-06-05 21:38:38,True,[Meditation ],NaN,"Jun 5, 2017, 2:30:01 PM",1,NaN,8,NaN,3,...,1.496655e+09,NaT,2017-06-05 14:30:01,NaN,NaT,3,08:00:00,NaT,08:00:00,1 days 00:00:00


In [43]:
dpd['MIGRAINESTART_TimeZone'].count()

40

In [44]:
dpd.loc[condition_dpd11, :].count()

dt_diary                  0
HADMIGRAINE               0
HELPMIGRAINETODAY         0
LURKINGMIGRAINE           0
MIGRAINEEND               0
MIGRAINESEVERITY          0
MIGRAINESTART             0
SLEEPDURATIONHOURS        0
SLEEPDURATIONMINUTES      0
SLEEPQUALITY              0
STRESSLEVEL               0
SYMPTOMSTODAY             0
TRIGGERSTODAY             0
dt_diary_tz               0
MIGRAINESTART_TimeZone    0
MIGRAINEEND_TimeZone      0
Extrappolated             0
ts_MSTART                 0
ts_MEND                   0
dt_MSTART                 0
dt_MEND                   0
ts_delta                  0
dt_delta                  0
mig_status                0
dt_deltasleep_hrs         0
dt_deltasleep_mins        0
dt_sleep                  0
dt_sleepQ                 0
dtype: int64

In [45]:
condition_dpd = ( (dpd['MIGRAINESTART_TimeZone'].notnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )

In [46]:
dpd.loc[condition_dpd, :].count()

dt_diary                  40
HADMIGRAINE               40
HELPMIGRAINETODAY         37
LURKINGMIGRAINE            0
MIGRAINEEND               40
MIGRAINESEVERITY          40
MIGRAINESTART             15
SLEEPDURATIONHOURS        40
SLEEPDURATIONMINUTES      12
SLEEPQUALITY              40
STRESSLEVEL               40
SYMPTOMSTODAY             40
TRIGGERSTODAY             39
dt_diary_tz               40
MIGRAINESTART_TimeZone    40
MIGRAINEEND_TimeZone      40
Extrappolated             25
ts_MSTART                 15
ts_MEND                   40
dt_MSTART                 15
dt_MEND                   40
ts_delta                  15
dt_delta                  15
mig_status                40
dt_deltasleep_hrs         40
dt_deltasleep_mins        12
dt_sleep                  40
dt_sleepQ                 40
dtype: int64

In [47]:
import glob
import pandas as pd

dir = 'C:/Users/Chisel-Lab/Documents/Migraine_ML/eda-explorer-master/sensordata/*.csv'
# C:\Users\Chisel-Lab\Documents\Migraine_ML\empaticaHRV-master\hrv_final
dir_hrv='C:/Users/Chisel-Lab/Documents/Migraine_ML/empaticaHRV-master/hrv_final/*.csv'
files = glob.glob(dir)
files_hrv = glob.glob(dir_hrv)
data = pd.DataFrame()


# for f in files:
# #     io.StringIO(df.to_csv()), index_col=0
# #     pd.read_csv(io.StringIO(f),index_col=0)
# #     newdata = pd.read_csv(f,index_col=0)
#     newdata = pd.read_csv(f, index_col=0, parse_dates=[0])
#     print(type(newdata.index[0]))

In [48]:
def computeAllAccelerometerFeatures(data, time_frames):
    if DEBUG: print("\t\tcomputing motion...")
    motion = computeMotion(data['AccelX'], data['AccelY'], data['AccelZ'])

    if DEBUG: print("\t\tcomputing steps...")
    steps = computeSteps(motion)

    if DEBUG: print("\t\tcomputing stillness...")
    stillness = computeStillness(motion)

    features = []

    for time_frame in time_frames:
        start = time_frame[0]
        print(start)
        end = time_frame[1]
        start1Hz = int(start / SAMPLING_RATE)
        end1Hz = end if end == -1 else int(end / SAMPLING_RATE)
        if DEBUG: print("\t\tcomputing features for time frame. Start index: "+ str(start)+ " end index: "+ str(end))

        time_frame_feats = computeAccelerometerFeaturesOverOneTimeFrame(motion[start:end],
                                                                        steps[start:end],
                                                                        stillness[start1Hz:end1Hz])
        features.append(time_frame_feats)

    return features, steps, motion

def computeMotion(acc1, acc2, acc3):
    '''Aggregates 3-axis accelerometer signal into a single motion signal'''
    return np.sqrt(np.array(acc1)**2 + np.array(acc2)**2 + np.array(acc3)**2)

def computeSteps(motion):
    '''Determines the location of steps from the aggregated accelerometer signal.
    Signal is low-pass filtered, then minimums are located in the signal. For each
    min, if the max absolute derivative (first difference) immediately surrounding
    it is greater than a threshold, it is counted as a step.

    Args:
        motion:		root mean squared 3 axis acceleration
    Returns:
        steps:		binary array at 8Hz which is 1 everywhere there is a step'''

    filtered_signal = filterSignalFIR(motion, 2, 256)
    diff = filtered_signal[1:]-filtered_signal[:-1]

    mins = scisig.argrelextrema(filtered_signal, np.less)[0]

    steps = [0] * len(filtered_signal)
    for m in mins:
        if m <= 4 or m >= len(diff) - 4:
            continue
        if max(abs(diff[m-4:m+4])) > STEP_DIFFERENCE_THRESHOLD:
            steps[m] = 1.0

    return steps

def filterSignalFIR(eda, cutoff=0.4, numtaps=64):
    f = cutoff/(SAMPLING_RATE/2.0)
    FIR_coeff = scisig.firwin(numtaps,f)

    return scisig.lfilter(FIR_coeff,1,eda)

def computeStillness(motion):
    '''Locates periods in which the person is still or motionless.
    Total acceleration must be less than a threshold for 95 percent of one
    minute in order for that minute to count as still

    Args:
        motion:		an array containing the root mean squared acceleration
    Returns:
        A 1Hz array that is 1 for each second belonging to a still period, 0 otherwise
    '''
    diff = motion[1:]-motion[:-1]
    momentary_stillness = diff < STILLNESS_MOTION_THRESHOLD
    np.append(momentary_stillness,0) # to ensure list is the same size as the full day signal
    num_minutes_in_day = 24*60

    #create array indicating whether person was still or not for each second of the day
    #to be still the momentary_stillness signal must be true for more than 95% of the minute
    #containing that second
    second_stillness = [0]*SECONDS_IN_DAY

    for i in range(num_minutes_in_day):
        hours_start = int(i / 60)
        mins_start = i % 60
        hours_end = int((i+1) / 60)
        mins_end = (i+1) % 60

        start_idx = getIndexFromTimestamp(hours_start, mins_start)
        end_idx = getIndexFromTimestamp(hours_end, mins_end)

        this_minute = momentary_stillness[start_idx:end_idx]
        minute_stillness = sum(this_minute) > PERCENT_STILLNESS_THRESHOLD*(60*SAMPLING_RATE)

        second_idx = int(start_idx/8)
        for si in range(second_idx,second_idx+60):
            second_stillness[si] = float(minute_stillness)

    return second_stillness

def computeAccelerometerFeaturesOverOneTimeFrame(motion, steps, stillness):
    ''' Computes all available features for a time period. Incoming signals are assumed to be from
    only that time period.

    Args:
        motion:						8Hz		root mean squared 3 axis acceleration
        steps:						8Hz		binary signal that is 1 if there is a step
        stillness:					1Hz		1 if the person was still during this second, 0 otherwise
    Returns:
        A list of features containing (in order):
        -Step count 								number of steps detected
        -mean step time during movement				average number of samples between two steps (aggregated first to 1 minute,
                                                    then we take the mean of only the parts of this signal occuring during movement)
        -percent stillness 							percentage of time the person spent nearly motionless
    '''

    features = []

    features.extend(computeStepFeatures(steps,stillness))
    features.append(countStillness(stillness))

    return features

def computeStepFeatures(steps,stillness):
    '''Counts the total number of steps over a given period,
    as well as the average time between steps (meant to approximate walking speed)

    Args:
        steps:	an binary array at 8 Hz that is 1 every time there is a step
    Returns:
        sum: 			the number of steps in a period
        median time: 	average number of samples between two steps'''

    sum_steps = float(sum(steps))

    step_indices = np.nonzero(steps)[0]
    diff = step_indices[1:]-step_indices[:-1]

    #ensure length of step difference array is the same so we can get the actual locations of step differences
    timed_step_diff = np.empty(len(steps)) * np.nan
    timed_step_diff[step_indices[:len(diff)]] = diff

    signal_length_1s = len(stillness)
    signal_length_1min = int(signal_length_1s / 60)

    # if there aren't enough steps during this period, cannot accurately compute mean step diff
    if len(timed_step_diff) < signal_length_1min:
        return [sum_steps, np.nan]

    agg_stillness = aggregateSignal(stillness, signal_length_1min, 'max')
    agg_step_diff = aggregateSignal(timed_step_diff, signal_length_1min, 'mean')

    movement_indices = [i for i in range(len(agg_stillness)) if agg_stillness[i] == 0.0]
    step_diff_during_movement = agg_step_diff[movement_indices]

    return [sum_steps,round(np.nanmean(step_diff_during_movement),10)]

def countStillness(stillness):
    '''Counts the total percentage of time spent still over a period

    Args:
        stillness:	an binary array at 1Hz that is 1 if that second is part of a still period
    Returns:
        the percentage time spent still over a period'''

    return float(sum(stillness)) / float(len(stillness))

def aggregateSignal(signal, new_signal_length, agg_method='sum'):
    new_signal = np.zeros(new_signal_length)
    samples_per_bucket = int(len(signal) / new_signal_length)

    #the new signal length must be large enough that there is at least 1 sample per bucket
    assert(samples_per_bucket > 0)

    for i in range(new_signal_length):
        if agg_method == 'sum':
            new_signal[i] = np.nansum(signal[i*samples_per_bucket:(i+1)*samples_per_bucket])
        elif agg_method == 'percent':
            new_signal[i] = np.nansum(signal[i*samples_per_bucket:(i+1)*samples_per_bucket]) / samples_per_bucket
        elif agg_method == 'mean':
            new_signal[i] = np.nanmean(signal[i*samples_per_bucket:(i+1)*samples_per_bucket])
        elif agg_method == 'max':
            new_signal[i] = np.nanmax(signal[i*samples_per_bucket:(i+1)*samples_per_bucket])
    return new_signal

def getIndexFromTimestamp(hours, mins=0): 
    return ((hours * 60) + mins) * 60 * SAMPLING_RATE

def inputTimeFrames():
    '''Allows user to choose the time frames over which they compute accelerometer features.'''

    time_frames = []
    print("Accelerometer features can be extracted over different time periods.")
    cont = 'y' #get_user_input("If you would like to enter a time period over which to compute features, enter 'y', or press enter to compute features over the entire file.")
#    while cont != 'y' or cont != 'Y':
#        start = int(get_user_input("Enter the starting hour of the time period (hour 0 is when the file starts):"))
#        end = int(get_user_input("Enter the ending hour of the time period (hour 0 is when the file starts; use -1 for the end of the file):"))
#        start = getIndexFromTimestamp(int(start))
#        if end != -1:
#            end = getIndexFromTimestamp(int(end))
#        time_frames.append([start,end])
#        print("Great! Now computing features for the following time periods:"+ str(time_frames))
#        cont = get_user_input("To add another time period, enter 'y'. To finish, press enter.")

    if len(time_frames) == 0:
        time_frames = [[0,-1]] # the whole file

    return time_frames

def saveFeaturesToFile(features, time_frames, output_file):
    of = open(output_file, 'w')
    of.write("Time period start hour, Time period end hour, Step count, Mean step time during movement, Percent stillness\n")
    tf_i = 0
    for tf in time_frames:
        output_str = str(tf[0]) + ' , ' + str(tf[1])
        for feat in features[tf_i]:
            output_str += ' , ' + str(feat)
        tf_i += 1
        of.write(output_str + '\n')
    of.close()
    print("Saved features to file"+ output_file)

In [49]:
time_frames = inputTimeFrames()


Accelerometer features can be extracted over different time periods.


In [50]:
import numpy as np
import pandas as pd
import scipy.signal as scisig
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

DEBUG = True

SAMPLING_RATE = 8

ONE_MINUTE_S = 60
THIRTY_MIN_S = ONE_MINUTE_S*30
SECONDS_IN_DAY = 24*60*60

STILLNESS_MOTION_THRESHOLD = .1
PERCENT_STILLNESS_THRESHOLD = .95

STEP_DIFFERENCE_THRESHOLD = 0.3


In [51]:

df_weather = pd.read_csv('weatherdata_hourly_everyday.csv', index_col=0, parse_dates=[0])



In [52]:
df_weather

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,precipType,pressure,summary,...,uvIndex,visibility,windBearing,windGust,windSpeed,delta_temperature,delta_humidity,delta_pressure,delta_cloudCover,delta_visibility
date,,,,,,,,,,,,,,,,,,,,,
2017-04-05 00:00:00-04:00,39.69,1.00,37.93,0.93,rain,0.0448,0.94,rain,1006.6,Light Rain,...,0,9.495,356.0,5.30,2.04,0.00,0.00,0.0,0.00,0.000
2017-04-05 01:00:00-04:00,39.52,1.00,37.79,0.93,rain,0.0087,0.65,rain,1007.2,Possible Drizzle,...,0,9.658,352.0,6.54,2.70,-0.17,0.00,0.6,0.00,0.163
2017-04-05 02:00:00-04:00,39.03,1.00,37.19,0.93,cloudy,0.0000,0.00,NaN,1007.5,Overcast,...,0,7.147,349.0,6.69,2.76,-0.49,0.00,0.3,0.00,-2.511
2017-04-05 03:00:00-04:00,36.43,1.00,36.74,0.93,cloudy,0.0000,0.00,NaN,1007.6,Overcast,...,0,6.020,354.0,7.36,3.17,-0.56,0.00,0.1,0.00,-1.127
2017-04-05 04:00:00-04:00,37.96,1.00,36.21,0.93,cloudy,0.0000,0.00,NaN,1008.3,Overcast,...,0,5.780,0.0,5.23,1.98,-0.51,0.00,0.7,0.00,-0.240
2017-04-05 05:00:00-04:00,37.55,1.00,35.74,0.93,cloudy,0.0000,0.00,NaN,1009.0,Overcast,...,0,8.623,6.0,4.96,2.00,-0.41,0.00,0.7,0.00,2.843
2017-04-05 06:00:00-04:00,37.23,1.00,35.42,0.93,cloudy,0.0000,0.00,NaN,1009.6,Overcast,...,0,8.965,12.0,4.62,1.77,-0.32,0.00,0.6,0.00,0.342
2017-04-05 07:00:00-04:00,36.99,1.00,35.07,0.93,cloudy,0.0000,0.00,NaN,1010.6,Overcast,...,0,9.645,6.0,5.14,1.92,-0.24,0.00,1.0,0.00,0.680
2017-04-05 08:00:00-04:00,37.22,1.00,35.15,0.92,cloudy,0.0000,0.00,NaN,1012.1,Overcast,...,1,8.771,2.0,4.68,1.90,0.23,-0.01,1.5,0.00,-0.874


In [53]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

objects = ('Python', 'C++', 'Java', 'Perl', 'Scala', 'Lisp')
y_pos = np.arange(len(objects))
performance = [10,8,6,4,2,1]

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Usage')
plt.title('Programming language usage')

plt.show()

In [54]:
dpd.head()

,dt_diary,HADMIGRAINE,HELPMIGRAINETODAY,LURKINGMIGRAINE,MIGRAINEEND,MIGRAINESEVERITY,MIGRAINESTART,SLEEPDURATIONHOURS,SLEEPDURATIONMINUTES,SLEEPQUALITY,...,ts_MEND,dt_MSTART,dt_MEND,ts_delta,dt_delta,mig_status,dt_deltasleep_hrs,dt_deltasleep_mins,dt_sleep,dt_sleepQ
0,2017-04-21 19:23:19,False,NaN,No,NaN,NaN,NaN,7,15,3,...,NaN,NaT,NaT,NaN,NaT,0,07:00:00,00:15:00,07:15:00,0 days 21:45:00
1,2017-04-21 20:45:56,False,[Meditation ],No,NaN,NaN,NaN,7,,3,...,NaN,NaT,NaT,NaN,NaT,0,07:00:00,00:00:00,07:00:00,0 days 21:00:00
2,2017-04-22 13:56:02,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaT,NaT,NaN,NaT,0,00:00:00,NaT,00:00:00,0 days 00:00:00
3,2017-04-22 23:15:33,True,NaN,NaN,"Apr 22, 2017, 11:14:59 PM",1,NaN,6,30,3,...,1.492885e+09,NaT,2017-04-22 23:14:59,NaN,NaT,3,06:00:00,00:30:00,06:30:00,0 days 19:30:00
4,2017-04-23 21:23:50,True,[Sleep],NaN,"Apr 23, 2017, 5:40:46 PM",2,NaN,7,45,4,...,1.492951e+09,NaT,2017-04-23 17:40:46,NaN,NaT,3,07:00:00,00:45:00,07:45:00,1 days 07:00:00


In [55]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
objects = ('Sleep Duration', 'stress level', 'migraine index')
y_pos = np.arange(len(objects))
performance = [int(dpd['SLEEPDURATIONHOURS'][3]),int(dpd['MIGRAINESEVERITY'][3]),int(dpd['SLEEPQUALITY'][3])]
# dpd['MIGRAINESEVERITY'][0] 
plt.bar(y_pos,performance, alpha=1,align='center')
plt.xticks(y_pos, objects)
plt.ylabel('Usage')
plt.title('Programming language usage')

plt.show()

In [56]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(ncols=2, nrows=7)
gs = axs[1, 1].get_gridspec()
# remove the underlying axes
for ax in axs[0:,-1]:
    ax.remove()
    
axbig = fig.add_subplot(gs[2:5, -1])
# axbig.annotate('Big Axes \nGridSpec[1:, -1]', (0.1, 0.5),
#                xycoords='axes fraction', va='center')
objects = ('Sleep Duration', 'stress level', 'migraine index')
y_pos = np.arange(len(objects))
performance = [int(dpd['SLEEPDURATIONHOURS'][3]),int(dpd['MIGRAINESEVERITY'][3]),int(dpd['SLEEPQUALITY'][3])]
axbig.bar(y_pos,performance, alpha=1,align='center')
axbig.set_xticks(y_pos, objects)
# axbig.set_ylabel('Usage')
# axbig.set_title('Programming language usage')
axs[0,0].barh(y_pos,performance, alpha=1,align='center')
axs[0,0].set_yticks(y_pos, objects)
# plt.sca(axs[0, 0])
# plt.xticks(y_pos, objects)
# axs[0,0].xticks(y_pos, objects)
# axs[0,0].set_ylabel('Usage')
# axs[0,0].set_title('Programming language usage') 
# fig.tight_layout()

plt.show()



<IPython.core.display.Javascript object>

In [57]:
# import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
objects = ('Sleep Duration', 'stress level', 'migraine index')
y_pos = np.arange(len(objects))
for i, row in dpd.loc[condition_dpd, :].iterrows() : 
    fig, axs = plt.subplots(1,figsize=(20, 10))
    performance = [int(row['SLEEPDURATIONHOURS']),int(row['MIGRAINESEVERITY']),int(row['SLEEPQUALITY'])]
    axs.barh(y_pos,performance, alpha=1,align='center')
    axs.set_yticks(y_pos, objects)
#     plt.xticks(y_pos, objects)
#     axs.set_ylabel('Migraine')
#     axs.title('Programming language usage')
#     axs.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
dpd=dpd.set_index('dt_diary')

In [59]:
mean = [0, 0]
cov = [[1, 1], [1, 2]]
x, y = np.random.multivariate_normal(mean, cov, 3000).T
fig = plt.figure(figsize=(20, 7))
grid = plt.GridSpec(8, 4, wspace=0.2, hspace=0.2)
left_bar=plt.subplot(grid[3, 0])
x0=plt.subplot(grid[0, 1:3])
x1=plt.subplot(grid[1, 1:3])
x2=plt.subplot(grid[2, 1:3])
x3=plt.subplot(grid[3, 1:3])
x4=plt.subplot(grid[4, 1:3])
x5=plt.subplot(grid[5, 1:3])
x6=plt.subplot(grid[6, 1:3])
right_bar=plt.subplot(grid[3, 3])
right_bar.plot(x, y, 'ok', markersize=3, alpha=0.2)

x6.plot(x, y, 'ok', markersize=3, alpha=0.2)

left_bar.plot(x, y, 'ok', markersize=3, alpha=0.2)
# plt.subplot(grid[1, :3])
# plt.subplot(grid[1, 2]);

C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


<IPython.core.display.Javascript object>

In [60]:
condition_actual = ( (dpd['MIGRAINESTART'].notnull() ) ) & ( dpd['MIGRAINEEND'].notnull() )
condition_extra = ( (dpd['MIGRAINESTART'].isnull() ) ) & ( dpd['MIGRAINEEND'].notnull() )


In [61]:
'2017-05-05' in dpd.index

True

In [62]:
# find 0 and 1 if in migraine or not
# if time is in start and end time make a column and add 1. 
# search this

from datetime import datetime

def isNowInTimePeriod(startTime, endTime, nowTime):
    if startTime < endTime:
        return nowTime >= startTime and nowTime <= endTime
    else: #Over midnight
        return nowTime >= startTime or nowTime <= endTime


timeStart = '3:00PM'
timeEnd = '11:00AM'
timeNow = '2:59AM'
timeEnd = datetime.strptime(timeEnd, "%I:%M%p")
timeStart = datetime.strptime(timeStart, "%I:%M%p")
timeNow = datetime.strptime(timeNow, "%I:%M%p")

print(isNowInTimePeriod(timeStart, timeEnd, timeNow))

True


In [240]:
#find hourly averages
datess=pd.unique(frame.index.date)
times=pd.unique(frame.index.time)

In [246]:
frame['time']=frame.index.time        

In [254]:
timearray=pd.unique(frame['time'])


In [293]:
frame['avgtemp']=0
frame['avgtonic']=0
frame['avgphasic']=0
frame['avgibi']=0
frame['avghrv']=0

In [263]:
data = pd.DataFrame([])
li=[]
for i in range(0,len(timearray)):
#     DF.append(SR_row,ignore_index=True)
    li.append(frame.loc[frame['time'] == timearray[i]].mean())

array([0, 0, 0, 0, 0, 0], dtype=int64)

In [329]:
frame.loc[frame['time'] == timearray[i]]['avgtemp']

index
2017-05-09 08:00:00-04:00    0
2017-05-10 08:00:00-04:00    0
2017-05-11 08:00:00-04:00    0
2017-05-12 08:00:00-04:00    0
2017-05-13 08:00:00-04:00    0
2017-05-14 08:00:00-04:00    0
Name: avgtemp, dtype: int64

In [316]:
(df1.loc[df1['time'] == timearray[i]]['Temp'].values[0])

30.487167989585387

In [295]:
frame.loc[frame['time'] == timearray[i]]

,AccelX,AccelY,AccelZ,EDA,HRV,IBI,SCR_width,Temp,amp,decay_time,...,phasic,rise_time,steps,tonic,time,avgtemp,avgtonic,avgphasic,avgibi,avghrv
index,,,,,,,,,,,,,,,,,,,,,
2017-05-09 08:00:00-04:00,0.440818,-0.305938,0.089986,6.682388,277.610050,0.492054,0.010620,30.037119,7.544099e-03,0.003323,...,3.970874,0.012308,0.033465,2.715223,08:00:00,0,0,0,0,0
2017-05-10 08:00:00-04:00,0.645842,-0.001862,0.315430,0.034516,411.421302,0.556023,0.000000,30.500510,7.336472e-07,0.000000,...,0.004660,0.000130,0.012531,0.029846,08:00:00,0,0,0,0,0
2017-05-11 08:00:00-04:00,0.672576,-0.082643,0.411954,0.043850,381.522394,0.578682,0.000052,29.911861,1.992988e-06,0.000013,...,0.002367,0.000188,0.004197,0.041483,08:00:00,0,0,0,0,0
2017-05-12 08:00:00-04:00,0.679748,-0.158360,0.284825,0.058554,403.521227,0.588382,0.000392,31.431155,1.082858e-05,0.000124,...,0.012074,0.000631,0.005662,0.046433,08:00:00,0,0,0,0,0
2017-05-13 08:00:00-04:00,0.669434,0.070796,0.348826,0.036819,417.891247,0.602880,0.000132,29.945125,1.876850e-06,0.000018,...,0.005189,0.000141,0.004803,0.031636,08:00:00,0,0,0,0,0
2017-05-14 08:00:00-04:00,0.524632,0.004552,0.467996,0.027000,404.514218,0.564487,0.000088,31.097237,7.245916e-07,0.000027,...,0.002900,0.000084,0.013473,0.024117,08:00:00,0,0,0,0,0


In [278]:
df1 = pd.DataFrame(li)

In [280]:
df1['time']=timearray

In [285]:
df1=df1.sort_values(by=['time'])

In [286]:
df1.time

19    00:00:00
20    01:00:00
21    02:00:00
22    03:00:00
23    04:00:00
0     05:00:00
1     06:00:00
2     07:00:00
3     08:00:00
4     09:00:00
5     10:00:00
6     11:00:00
7     12:00:00
8     13:00:00
9     14:00:00
10    15:00:00
11    16:00:00
12    17:00:00
13    18:00:00
14    19:00:00
15    20:00:00
16    21:00:00
17    22:00:00
18    23:00:00
Name: time, dtype: object

In [65]:
f=10
df = pd.read_csv(files[f], index_col=0, parse_dates=[0])
df_hrv = pd.read_csv(files_hrv[f], index_col=0, parse_dates=[0])
df_hrv = df_hrv.sort_index()
df = df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')
df = df.sort_index()
data=df[['AccelX','AccelY','AccelZ']]
features, steps, motion = computeAllAccelerometerFeatures(data, time_frames)
df["steps"] = steps
df["motion"] = motion
df=df.resample('1H').mean()
df_hrv=df_hrv.resample('1H').mean()
df['IBI']=df_hrv['IBI']
df['HRV']=df_hrv['HRV']
date=str(newdata.index[0].date())
df_w=df_weather.loc[date]
df_n=pd.concat([df, df_w], axis=1, sort=False)
df.to_csv(date+'.csv')
if(f==0):
    df_all=df
df_all.append(df2)

		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1


C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:181: RuntimeWarning: Mean of empty slice


In [228]:
import pandas as pd
import glob

path = r'HourlyAverageAllData' # use your path
path_output = r'hourlyaverage_output' # use your path
path_Selected = r'selectedata' # use your path
all_files = glob.glob(path + "/*.csv")
all_files_output=glob.glob(path_Selected + "/*.csv")
li = []

for filename in all_files_output:
    df = pd.read_csv(filename, index_col=0, parse_dates=[0])
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=False)

C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [230]:
selectedframe=frame[['Temp','EDA','HRV','IBI','motion','steps','phasic','output']]

In [234]:
selectedframe.to_csv('SelectedHourly_data_output_final.csv')

In [233]:
for filename in all_files:
    df = pd.read_csv(filename, index_col=0,parse_dates=[0])
    date=str(df.index[0].date())
    if date in dpd.index:
        row1=dpd.loc[date]
        if((len(row1)<2) & (len(row1)!=0) ):
            if (row1['HADMIGRAINE'].bool()):
                df['output']=0            
    #             period = pd.date_range(start=row1['MIGRAINESTART_TimeZone'].values,end=row1['MIGRAINEEND_TimeZone'].values,freq='25ms') 
                startTime=row1['MIGRAINESTART_TimeZone'].values
                endTime=row1['MIGRAINEEND_TimeZone'].values 
                df['index1']=df.index
                for i in range(0,len(df)):
                    df['output'][i]=isNowInTimePeriod(startTime, endTime,df['index1'].values[i])
            else:
                df['output']=0
    df.to_csv('hourlyaverage_output/'+date+'.csv')

139

In [224]:
frame['hour'] = frame.index.hour
frame.groupby(by=["hour"]).hour.mean()

hour
0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
Name: hour, dtype: int64

In [204]:
frame.to_csv('Hourly_data_output_final.csv')

In [160]:
frame['output'].sum()

387.0

In [187]:
frame = frame.drop('index1', 1)
frame = frame.drop('index', 1)



KeyError: "['index1'] not found in axis"

In [188]:
frame = frame.drop('index', 1)

In [178]:
frame = frame.drop('peak_end_times', 1)

In [180]:
frame = frame.drop('peak_start_times', 1)

In [194]:
frame = frame.drop('half_rise', 1)

In [191]:
frame.columns

Index(['AccelX', 'AccelY', 'AccelZ', 'EDA', 'HRV', 'IBI', 'SCR_width', 'Temp',
       'amp', 'decay_time', 'filtered_eda', 'half_rise', 'max_deriv', 'motion',
       'output', 'peak_end', 'peak_start', 'peaks', 'peaks_plot', 'phasic',
       'rise_time', 'steps', 'tonic'],
      dtype='object')

In [205]:
frame

,AccelX,AccelY,AccelZ,EDA,HRV,IBI,SCR_width,Temp,amp,decay_time,...,motion,output,peak_end,peak_start,peaks,peaks_plot,phasic,rise_time,steps,tonic
0,-0.096548,0.175509,0.758322,0.501601,245.802012,0.722960,0.000130,35.013322,0.000006,0.000039,...,0.975053,0.0,0.000208,0.005347,0.000208,0.001337,1.234282e-02,0.000495,0.000174,0.489187
1,-0.476092,0.344559,0.211559,0.520270,355.238078,0.673280,0.000078,34.795259,0.000001,0.000013,...,0.998980,0.0,0.000035,0.002743,0.000035,0.001325,8.110972e-03,0.000087,0.000451,0.512098
2,-0.122452,0.126608,0.284706,0.442489,350.539296,0.567168,0.002609,34.490477,0.000074,0.001108,...,1.011086,0.0,0.000941,0.005318,0.001085,0.026492,7.225020e-02,0.002207,0.012951,0.370287
3,0.557886,-0.039852,0.460277,0.273626,365.043371,0.553132,0.001033,31.003524,0.000031,0.000339,...,0.988851,1.0,0.000694,0.005451,0.000694,0.008867,3.881836e-02,0.001606,0.021910,0.234705
4,0.626886,-0.155061,0.234050,0.232334,359.308915,0.556886,0.001466,31.514834,0.000037,0.000592,...,0.995374,1.0,0.000848,0.006360,0.000742,0.012616,4.878072e-02,0.001515,0.011625,0.183637
5,0.513615,-0.084583,0.563406,0.248111,398.680364,0.568066,0.000923,31.615630,0.000033,0.000235,...,0.991712,1.0,0.000557,0.005048,0.000522,0.003145,3.975829e-02,0.001179,0.008773,0.208296
6,0.635910,-0.020912,0.468672,0.276163,392.074145,0.595397,0.001949,31.975752,0.000030,0.000786,...,0.984241,1.0,0.000729,0.005035,0.000764,0.004904,4.099700e-02,0.001688,0.003819,0.234979
7,0.581435,-0.012608,0.266603,0.242329,392.971313,0.593583,0.001702,32.130233,0.000042,0.000477,...,0.992405,1.0,0.000875,0.006754,0.000875,0.005616,4.165176e-02,0.002130,0.006299,0.200490
8,0.501524,0.111891,0.379065,0.269388,373.599302,0.562537,0.000827,31.751312,0.000036,0.000239,...,0.991469,1.0,0.000452,0.005778,0.000452,0.002490,3.524496e-02,0.001088,0.007344,0.234108
9,0.460252,-0.109838,0.608641,0.331980,361.052855,0.601538,0.000417,32.582346,0.000016,0.000104,...,0.981598,1.0,0.000347,0.006250,0.000347,0.002022,4.826652e-02,0.000690,0.005556,0.283656


In [199]:
frame['amp']

0     0.000006
1     0.000001
2     0.000074
3     0.000031
4     0.000037
5     0.000033
6     0.000030
7     0.000042
8     0.000036
9     0.000016
10    0.000035
11    0.000000
12    0.000000
13    0.000000
14    0.000049
15    0.000047
16    0.000003
17    0.000017
18    0.000001
19    0.000000
20    0.000002
21    0.000004
22    0.000001
23    0.000022
0     0.000544
1     0.000127
2     0.000045
3     0.000011
4     0.000032
5     0.000008
        ...   
4     0.000017
5     0.000070
6     0.000107
7     0.000784
8     0.001242
9     0.002655
10    0.000057
11    0.000120
12    0.000661
13    0.000023
14    0.000000
15    0.000006
16    0.001746
17    0.000226
18    0.000446
19    0.000893
20    0.001452
0     0.001435
1     0.000809
2     0.000152
3     0.003136
4     0.004247
5     0.000000
6     0.000000
7     0.000074
8     0.000609
9     0.001622
10    0.001317
11    0.000501
12    0.003987
Name: amp, Length: 1454, dtype: float64

In [202]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

frame.fillna(frame.mean())

pca = PCA(n_components=5)
pca.fit(frame)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [130]:

def isNowInTimePeriod(startTime, endTime, nowTime):
    if startTime < endTime:
        return nowTime >= startTime and nowTime <= endTime
    else: #Over midnight
        return nowTime >= startTime or nowTime <= endTime

delta = time1 - time2
df['index1']=df.index
for i in range(0,len(df)):
    df['output'][i]=isNowInTimePeriod(startTime, endTime,df['index1'].values[i])
#     newdata100['output32'][i]=TimeremaininginPeriod(startTime[0], endTime[0],newdata100['index1'].values[i])

C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
def TimeremaininginPeriod(startTime, endTime, nowTime):
    if nowTime < startTime:
        print(type(startTime))
        print(type(nowTime))
        return (startTime - nowTime)
    if nowTime > startTime: #Over midnight
        return (nowTime-nowTime) \
    

In [207]:
frame


,AccelX,AccelY,AccelZ,EDA,HRV,IBI,SCR_width,Temp,amp,decay_time,...,motion,output,peak_end,peak_start,peaks,peaks_plot,phasic,rise_time,steps,tonic
0,-0.096548,0.175509,0.758322,0.501601,245.802012,0.722960,0.000130,35.013322,0.000006,0.000039,...,0.975053,0.0,0.000208,0.005347,0.000208,0.001337,1.234282e-02,0.000495,0.000174,0.489187
1,-0.476092,0.344559,0.211559,0.520270,355.238078,0.673280,0.000078,34.795259,0.000001,0.000013,...,0.998980,0.0,0.000035,0.002743,0.000035,0.001325,8.110972e-03,0.000087,0.000451,0.512098
2,-0.122452,0.126608,0.284706,0.442489,350.539296,0.567168,0.002609,34.490477,0.000074,0.001108,...,1.011086,0.0,0.000941,0.005318,0.001085,0.026492,7.225020e-02,0.002207,0.012951,0.370287
3,0.557886,-0.039852,0.460277,0.273626,365.043371,0.553132,0.001033,31.003524,0.000031,0.000339,...,0.988851,1.0,0.000694,0.005451,0.000694,0.008867,3.881836e-02,0.001606,0.021910,0.234705
4,0.626886,-0.155061,0.234050,0.232334,359.308915,0.556886,0.001466,31.514834,0.000037,0.000592,...,0.995374,1.0,0.000848,0.006360,0.000742,0.012616,4.878072e-02,0.001515,0.011625,0.183637
5,0.513615,-0.084583,0.563406,0.248111,398.680364,0.568066,0.000923,31.615630,0.000033,0.000235,...,0.991712,1.0,0.000557,0.005048,0.000522,0.003145,3.975829e-02,0.001179,0.008773,0.208296
6,0.635910,-0.020912,0.468672,0.276163,392.074145,0.595397,0.001949,31.975752,0.000030,0.000786,...,0.984241,1.0,0.000729,0.005035,0.000764,0.004904,4.099700e-02,0.001688,0.003819,0.234979
7,0.581435,-0.012608,0.266603,0.242329,392.971313,0.593583,0.001702,32.130233,0.000042,0.000477,...,0.992405,1.0,0.000875,0.006754,0.000875,0.005616,4.165176e-02,0.002130,0.006299,0.200490
8,0.501524,0.111891,0.379065,0.269388,373.599302,0.562537,0.000827,31.751312,0.000036,0.000239,...,0.991469,1.0,0.000452,0.005778,0.000452,0.002490,3.524496e-02,0.001088,0.007344,0.234108
9,0.460252,-0.109838,0.608641,0.331980,361.052855,0.601538,0.000417,32.582346,0.000016,0.000104,...,0.981598,1.0,0.000347,0.006250,0.000347,0.002022,4.826652e-02,0.000690,0.005556,0.283656


In [206]:
dpd

,HADMIGRAINE,HELPMIGRAINETODAY,LURKINGMIGRAINE,MIGRAINEEND,MIGRAINESEVERITY,MIGRAINESTART,SLEEPDURATIONHOURS,SLEEPDURATIONMINUTES,SLEEPQUALITY,STRESSLEVEL,...,ts_MEND,dt_MSTART,dt_MEND,ts_delta,dt_delta,mig_status,dt_deltasleep_hrs,dt_deltasleep_mins,dt_sleep,dt_sleepQ
dt_diary,,,,,,,,,,,,,,,,,,,,,
2017-04-21 19:23:19,False,NaN,No,NaN,NaN,NaN,7,15,3,2,...,NaN,NaT,NaT,NaN,NaT,0,07:00:00,00:15:00,07:15:00,0 days 21:45:00
2017-04-21 20:45:56,False,[Meditation ],No,NaN,NaN,NaN,7,,3,2,...,NaN,NaT,NaT,NaN,NaT,0,07:00:00,00:00:00,07:00:00,0 days 21:00:00
2017-04-22 13:56:02,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaT,NaT,NaN,NaT,0,00:00:00,NaT,00:00:00,0 days 00:00:00
2017-04-22 23:15:33,True,NaN,NaN,"Apr 22, 2017, 11:14:59 PM",1,NaN,6,30,3,2,...,1.492885e+09,NaT,2017-04-22 23:14:59,NaN,NaT,3,06:00:00,00:30:00,06:30:00,0 days 19:30:00
2017-04-23 21:23:50,True,[Sleep],NaN,"Apr 23, 2017, 5:40:46 PM",2,NaN,7,45,4,3,...,1.492951e+09,NaT,2017-04-23 17:40:46,NaN,NaT,3,07:00:00,00:45:00,07:45:00,1 days 07:00:00
2017-04-24 23:16:57,False,NaN,No,NaN,NaN,NaN,6,30,4,2,...,NaN,NaT,NaT,NaN,NaT,0,06:00:00,00:30:00,06:30:00,1 days 02:00:00
2017-04-25 22:15:56,True,[Meditation ],NaN,"Apr 25, 2017, 10:15:20 PM",1,NaN,6,45,4,2,...,1.493141e+09,NaT,2017-04-25 22:15:20,NaN,NaT,3,06:00:00,00:45:00,06:45:00,1 days 03:00:00
2017-04-26 21:17:02,False,NaN,Yes,NaN,NaN,NaN,7,30,4,2,...,NaN,NaT,NaT,NaN,NaT,0,07:00:00,00:30:00,07:30:00,1 days 06:00:00
2017-04-27 23:11:28,False,NaN,Yes,NaN,NaN,NaN,7,45,3,2,...,NaN,NaT,NaT,NaN,NaT,0,07:00:00,00:45:00,07:45:00,0 days 23:15:00


In [ ]:
if nowTime < startTime:
    x=(startTime - nowTime)
if nowTime > startTime & nowTime<endTime: #Over midnight
    x=(nowTime-nowTime) 
if nowTime == endTime:
   # update start and end times
    

In [129]:
df['output']

index
2017-05-01 00:00:00-04:00    0
2017-05-01 01:00:00-04:00    0
2017-05-01 02:00:00-04:00    0
2017-05-01 03:00:00-04:00    1
2017-05-01 04:00:00-04:00    1
2017-05-01 05:00:00-04:00    1
2017-05-01 06:00:00-04:00    1
2017-05-01 07:00:00-04:00    1
2017-05-01 08:00:00-04:00    1
2017-05-01 09:00:00-04:00    1
2017-05-01 10:00:00-04:00    1
2017-05-01 11:00:00-04:00    1
2017-05-01 12:00:00-04:00    1
2017-05-01 13:00:00-04:00    1
2017-05-01 14:00:00-04:00    1
2017-05-01 15:00:00-04:00    1
2017-05-01 16:00:00-04:00    1
2017-05-01 17:00:00-04:00    1
2017-05-01 18:00:00-04:00    0
2017-05-01 19:00:00-04:00    0
2017-05-01 20:00:00-04:00    0
2017-05-01 21:00:00-04:00    0
2017-05-01 22:00:00-04:00    0
2017-05-01 23:00:00-04:00    0
Name: output, dtype: int64

In [133]:
for filename in all_files:
    df = pd.read_csv(filename, index_col=0,parse_dates=[0])
    date=str(df.index[0].date())
    if date in dpd.index:
        row1=dpd.loc[date]
        if((len(row1)<2) & (len(row1)!=0) ):
            if (row1['HADMIGRAINE'].bool()):
                df['output']=0            
    #             period = pd.date_range(start=row1['MIGRAINESTART_TimeZone'].values,end=row1['MIGRAINEEND_TimeZone'].values,freq='25ms') 
                startTime=row1['MIGRAINESTART_TimeZone'].values
                endTime=row1['MIGRAINEEND_TimeZone'].values 
                df['index1']=df.index
                for i in range(0,len(df)):
                    df['output'][i]=isNowInTimePeriod(startTime, endTime,df['index1'].values[i])
            else:
                df['output']=0
    df.to_csv('hourlyaverage_output/'+date+'.csv')

C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [134]:
df['output']

index
2017-07-21 00:00:00-04:00    1
2017-07-21 01:00:00-04:00    1
2017-07-21 02:00:00-04:00    1
2017-07-21 03:00:00-04:00    1
2017-07-21 04:00:00-04:00    1
2017-07-21 05:00:00-04:00    1
2017-07-21 06:00:00-04:00    1
2017-07-21 07:00:00-04:00    1
2017-07-21 08:00:00-04:00    1
2017-07-21 09:00:00-04:00    0
2017-07-21 10:00:00-04:00    0
2017-07-21 11:00:00-04:00    0
2017-07-21 12:00:00-04:00    0
Name: output, dtype: int64

In [85]:
for f in range(0,len(files)):
    df = pd.read_csv(files[f], index_col=0, parse_dates=[0])
    df_hrv = pd.read_csv(files_hrv[f], index_col=0, parse_dates=[0])
    df_hrv = df_hrv.sort_index()
    df = df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')
    df = df.sort_index()
    data=df[['AccelX','AccelY','AccelZ']]
    features, steps, motion = computeAllAccelerometerFeatures(data, time_frames)
    df["steps"] = steps
    df["motion"] = motion
    df=df.resample('1H').mean()
    df_hrv=df_hrv.resample('1H').mean()
    df['IBI']=df_hrv['IBI']
    df['HRV']=df_hrv['HRV']
    date=str(df.index[0].date())
    df_w=df_weather.loc[date]
    df_n=pd.concat([df, df_w], axis=1, sort=False)
    df_n.to_csv('HourlyAverageAllData/'+date+'.csv')
    if(f==0):
        df_all=df
    df_all.append(df)


		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1


C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:181: RuntimeWarning: Mean of empty slice
C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1


C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:181: RuntimeWarning: Mean of empty slice
C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1
		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1


...

		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1
		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1
		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1
		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1


C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:181: RuntimeWarning: Mean of empty slice
C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1


C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:181: RuntimeWarning: Mean of empty slice
C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1


In [86]:

frames = [ process_your_file(f) for f in files ]
result = pd.concat(frames)

,EDA,filtered_eda,AccelX,AccelY,AccelZ,Temp,peaks,peak_start,peak_end,amp,...,rise_time,decay_time,SCR_width,phasic,tonic,peaks_plot,steps,motion,IBI,HRV
index,,,,,,,,,,,,,,,,,,,,,
2017-05-01 00:00:00-04:00,0.501601,0.501539,-0.096548,0.175509,0.758322,35.013322,0.000208,0.005347,0.000208,0.000006,...,0.000495,0.000039,0.000130,1.234282e-02,0.489187,0.001337,0.000174,0.975053,0.722960,245.802012
2017-05-01 01:00:00-04:00,0.520270,0.520218,-0.476092,0.344559,0.211559,34.795259,0.000035,0.002743,0.000035,0.000001,...,0.000087,0.000013,0.000078,8.110972e-03,0.512098,0.001325,0.000451,0.998980,0.673280,355.238078
2017-05-01 02:00:00-04:00,0.442489,0.442536,-0.122452,0.126608,0.284706,34.490477,0.001085,0.005318,0.000941,0.000074,...,0.002207,0.001108,0.002609,7.225020e-02,0.370287,0.026492,0.012951,1.011086,0.567168,350.539296
2017-05-01 03:00:00-04:00,0.273626,0.273508,0.557886,-0.039852,0.460277,31.003524,0.000694,0.005451,0.000694,0.000031,...,0.001606,0.000339,0.001033,3.881836e-02,0.234705,0.008867,0.021910,0.988851,0.553132,365.043371
2017-05-01 04:00:00-04:00,0.232334,0.232448,0.626886,-0.155061,0.234050,31.514834,0.000742,0.006360,0.000848,0.000037,...,0.001515,0.000592,0.001466,4.878072e-02,0.183637,0.012616,0.011625,0.995374,0.556886,359.308915
2017-05-01 05:00:00-04:00,0.248111,0.248077,0.513615,-0.084583,0.563406,31.615630,0.000522,0.005048,0.000557,0.000033,...,0.001179,0.000235,0.000923,3.975829e-02,0.208296,0.003145,0.008773,0.991712,0.568066,398.680364
2017-05-01 06:00:00-04:00,0.276163,0.275994,0.635910,-0.020912,0.468672,31.975752,0.000764,0.005035,0.000729,0.000030,...,0.001688,0.000786,0.001949,4.099700e-02,0.234979,0.004904,0.003819,0.984241,0.595397,392.074145
2017-05-01 07:00:00-04:00,0.242329,0.242133,0.581435,-0.012608,0.266603,32.130233,0.000875,0.006754,0.000875,0.000042,...,0.002130,0.000477,0.001702,4.165176e-02,0.200490,0.005616,0.006299,0.992405,0.593583,392.971313
2017-05-01 08:00:00-04:00,0.269388,0.269344,0.501524,0.111891,0.379065,31.751312,0.000452,0.005778,0.000452,0.000036,...,0.001088,0.000239,0.000827,3.524496e-02,0.234108,0.002490,0.007344,0.991469,0.562537,373.599302


In [64]:
#weather + stress leverl addition
for f in range(0,len(files)):
    df = pd.read_csv(files[f], index_col=0, parse_dates=[0])
    df_hrv = pd.read_csv(files_hrv[f], index_col=0, parse_dates=[0])
    newdata_hrv = df_hrv.sort_index()
    newdata1 = df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')
    newdata1 = newdata1.sort_index()
    newdata10=newdata1.resample('10t').mean()
    newdata=newdata1.resample('1T').mean()
    data=newdata1[['AccelX','AccelY','AccelZ']]
    features, steps, motion = computeAllAccelerometerFeatures(data, time_frames)
    data["steps"] = steps
    data["motion"] = motion
    condition_dpd = ( (dpd['MIGRAINESTART_TimeZone'].notnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )
#     newdata = newdata.drop_duplicates(subset=, keep='first')
    date=str(newdata.index[0].date())
    df_wthr=df_weather.loc[date]
    print(date)
#     fig, axs = plt.subplots(2, 9, sharex=0,gridspec_kw={'hspace': 0, 'wspace': 0})
    fig, axs = plt.subplots(ncols=4,nrows=9,figsize=(20, 10))
    gs = axs[1, 1].get_gridspec()
# remove the underlying axes

    axbig = fig.add_subplot(gs[3:5, 3])
    axtable = fig.add_subplot(gs[5:7, 3])
    objects = ('Sleep Duration', 'stress level', 'migraine index')
    y_pos = np.arange(len(objects))
    fig.suptitle('Sensor Data Plots '+date) 
    x_pos = np.arange(len(objects))
    if date in dpd.index:
        row1=dpd.loc[date]

        if((len(row1)<2) & (len(row1)!=0) ):
            performance = [int(row1['SLEEPDURATIONHOURS']),int(row1['MIGRAINESEVERITY']),int(row1['SLEEPQUALITY'])]
            axbig.set_xticks(x_pos)
            axbig.set_xticklabels(('Sleep Duration', 'Stress Level', 'Migraine'))
            axbig.bar(x_pos,performance, alpha=1,align='center')
            newdf=pd.DataFrame()
            for x in dpd.loc[date]["TRIGGERSTODAY"].values:
                newdf['TRIGGERSTODAY']=pd.Series(x)
            for x in dpd.loc[date]["HELPMIGRAINETODAY"].values:
                newdf['HELPMIGRAINETODAY']=pd.Series(x)
            for x in dpd.loc[date]["SYMPTOMSTODAY"].values:
                newdf['SYMPTOMSTODAY']=pd.Series(x)
            # Hide axes
            axtable.xaxis.set_visible(False) 
            axtable.yaxis.set_visible(False)
            axtable.axis('off')
            collabel=("TRIGGERSTODAY", "HELPMIGRAINETODAY", "SYMPTOMSTODAY")
            axtable.table(cellText=newdf.values,colLabels=collabel)
        else: 
            print('not in it')
    
#     for i, row in dpd.loc[condition_actual, :].iterrows() : 
#         for i in range(9):
#             axs[i][0:2].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
#             #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
#             axs[i][0:2].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
#             axs[i][0:2].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
#             axs[i][0:2].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
#     for i, row in dpd.loc[condition_extra, :].iterrows() : 
#         for i in range(9):
#             axs[i][0:2].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
#             #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

#             axs[i][0:2].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
#             axs[i][0:2].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
#             axs[i][0:2].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)

    
    axs[0][0:2].plot(newdata10.index, newdata10['Temp'],label='Body Temperature')
    #                axs[1].plot(peaks.index,peaks['peaks_plot'],'#4DBD33')
    axs[1][0:2].plot(newdata1.index,5*newdata1['peaks_plot'],'#4DBD33')
    axs[1][0:2].plot(newdata.index, newdata['filtered_eda'],label='Phasic')
    axs[1][0:2].plot(newdata.index,newdata['tonic'],label='Tonic')
    axs[2][0:2].plot(newdata.index, newdata['AccelX'],newdata.index, newdata['AccelY'],newdata.index, newdata['AccelZ'])
    axs[3][0:2].plot(data.index, (3*data['steps']),data.index, data['motion'])
    axs[4][0:2].plot(newdata_hrv.index, newdata_hrv['IBI'])
    axs[5][0:2].plot(newdata_hrv.index, newdata_hrv['HRV'])
    axs[8][0:2].plot(df_wthr.index, df_wthr['apparentTemperature'],label='Atmospheric Temperature')
    axs[6][0:2].plot(df_wthr.index, df_wthr['humidity'],label='Humidity')
    axs[7][0:2].plot(df_wthr.index, df_wthr['pressure'],label='Pressure')
    axs[6][0:2].plot(df_wthr.index, df_wthr['cloudCover'],label='Cloud Cover')
    delta_t=axs[8][0:2].twinx()
    delta_t.plot(df_wthr.index, df_wthr['deltaTemp_1'],'c-',label='Delta Temperature')
    delta_h_cc=axs[6][0:2].twinx()
    delta_h_cc.plot(df_wthr.index, df_wthr['deltahumidity_1'],'g-',label='Delta Humidity')
    
    delta_h_cc.plot(df_wthr.index, df_wthr['deltacloudCover_1'],'m-',label='Delta Cloud Cover')
    delta_p=axs[7][0:2].twinx()
    delta_p.plot(df_wthr.index, df_wthr['delta_pressure'],'r-',label='Delta Pressure')
    delta_h_cc.set_ylabel('delta H & CC',color='g',fontsize='x-small')
    delta_p.set_ylabel('delta Pressure',color='r',fontsize='x-small')
    delta_t.set_ylabel('delta Temp',color='c',fontsize='x-small')
    
# #     plt.legend()
#     axs[0][0].legend(fontsize='xx-small')
#     axs[1][0].legend(fontsize='xx-small')
#     axs[2][0].legend(fontsize='xx-small')
#     axs[3][0].legend(fontsize='xx-small')
#     axs[4][0].legend(fontsize='xx-small')
#     axs[5][0].legend(fontsize='xx-small')
#     axs[6][0].legend(fontsize='xx-small')
#     axs[7][0].legend(fontsize='xx-small')
#     #            plt.xticks( rotation=25 )
#     axs[0][0].set_ylabel('Temperature',fontsize='x-small')
# #     axs[0].set_ylim (24 ,50)
#     axs[1][0].set_ylabel('EDA',fontsize='x-small')
#     data_min = min(newdata['EDA'])
#     data_max =5
#     axs[1][0].set_ylim(data_min ,data_max)
#     axs[2][0].set_ylabel('Accelerometer',fontsize='x-small')
#     axs[3][0].set_ylabel('Motion/Steps',fontsize='x-small')
#     axs[4][0].set_ylabel('IBI',fontsize='x-small')
#     axs[5][0].set_ylabel('HRV',fontsize='x-small')
#     axs[8][0].set_ylabel('Temperature',fontsize='x-small')
#     axs[6][0].set_ylabel('Hum & CC',fontsize='x-small')
#     axs[7][0].set_ylabel('Pressure',fontsize='x-small')
# #     axs[8].set_ylabel('CloudCover')
#     axs[8][0].set_xlabel('Time',fontsize='x-small')
    for ax in axs[0:,3]:
        
        ax.remove()
    
    # Hide x labels and tick labels for all but bottom plot.
    
    for ax in axs[0:,0:2]:
       ax.label_outer()
       hours = mdates.HourLocator(interval = 6)
       h_fmt = mdates.DateFormatter('%I:%M %p')
       ax.set_xlim([date+" 00:00:00-04:00", date+" 23:59:59-04:00"])
       #     ax.set_xlim(["00:00:00", "23:59:59"])
       xax = ax.get_xaxis()
       xax.set_major_locator(hours)
       xax.set_major_formatter(h_fmt)
       xax.set_minor_locator(mdates.HourLocator(interval = 1))
       xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
       xax.set_tick_params(which='major', pad=15)
    plt.tight_layout()
#     plt.show()
    plt.savefig('subplots_sensordata/test2/'+date+".png",bbox_inches="tight",dpi = 300)
#     plt.close()
#     axs1.savefig('subplots_sensordata/ '+date+"-da3.png",bbox_inches="tight",dpi = 900)
        

C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec

widths = [5, 1]
heights = [10,1,1,1,1,1,1,1,1]
gs_kw = dict(width_ratios=widths, height_ratios=heights)
fig, axs = plt.subplots(ncols=2 nrows=9,figsize=(20, 10),constrained_layout=True,gridspec_kw=gs_kw)
gs = axs[1, 1].get_gridspec()
# gs = matplotlib.gridspec.GridSpec(9,2,width_ratios=[4,1.4])

# fig , axs = plt.subplot(gs)
# ax1 = fig.add_subplot(gs[:,0])
# ax2 = fig.add_subplot(gs[1,1])

plt.show()


In [ ]:
fig5 = plt.figure(constrained_layout=True)
widths = [5, 1]
heights = [10,1,1,1,1,1,1,1,1]
spec5 = fig5.add_gridspec(ncols=2, nrows=9, width_ratios=widths,
                          height_ratios=heights)
for row in range(9):
    for col in range(2):
        ax = fig5.add_subplot(spec5[row, col])
        label = 'Width: {}\nHeight: {}'.format(widths[col], heights[row])
        ax.annotate(label, (0.1, 0.5), xycoords='axes fraction', va='center')

In [ ]:
dpd.loc['2017-05-07']

In [287]:
#Hourly average ibi + EDA + Activity + (no temperature)
for f in range(0,len(files)):
    df = pd.read_csv(files[f], index_col=0, parse_dates=[0])
    df_hrv = pd.read_csv(files_hrv[f], index_col=0, parse_dates=[0])
    newdata_hrv = df_hrv.sort_index()
    newdata1 = df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')
    newdata1 = newdata1.sort_index()
    newdata_hrv=newdata_hrv.resample('1H').mean()
    newdata10=newdata1.resample('1H').mean()
    newdata=newdata1.resample('1T').mean()
    data=newdata1[['AccelX','AccelY','AccelZ']]
    features, steps, motion = computeAllAccelerometerFeatures(data, time_frames)
    data["steps"] = steps
    data["motion"] = motion
    condition_dpd = ( (dpd['MIGRAINESTART_TimeZone'].notnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )
#     newdata = newdata.drop_duplicates(subset=, keep='first')
    date=str(newdata.index[0].date())
    df_wthr=df_weather.loc[date]
    print(date)
#     fig, axs = plt.subplots(2, 9, sharex=0,gridspec_kw={'hspace': 0, 'wspace': 0})

    fig, axs = plt.subplots(ncols=2,nrows=4,figsize=(20, 10),gridspec_kw={'width_ratios' : [4,1]})
    gs = axs[1, 1].get_gridspec()
    
# remove the underlying axes

    axbig = fig.add_subplot(gs[0:2, 1])
    axtable = fig.add_subplot(gs[2:3, 1])
    objects = ('Sleep Duration', 'stress level', 'migraine index')
    x_pos = np.arange(len(objects))
#     fig.suptitle('Sensor Data Plots '+date) 
    y_pos = np.arange(len(objects))
    if date in dpd.index:
        row1=dpd.loc[date]

        if((len(row1)<2) & (len(row1)!=0) ):
            performance = [int(row1['SLEEPDURATIONHOURS']),int(row1['MIGRAINESEVERITY']),int(row1['SLEEPQUALITY'])]
            axbig.set_xticks(x_pos)
            axbig.set_xticklabels(('Sleep Duration','Migraine Severity','Stress Level'))
            axbig.bar(x_pos,performance, alpha=1,align='center')
            axbig.set_ylim(0,9)
            newdf=pd.DataFrame()
            for x in dpd.loc[date]["TRIGGERSTODAY"].values:
                newdf['TRIGGERSTODAY']=pd.Series(x)
            for x in dpd.loc[date]["HELPMIGRAINETODAY"].values:
                newdf['HELPMIGRAINETODAY']=pd.Series(x)
            for x in dpd.loc[date]["SYMPTOMSTODAY"].values:
                newdf['SYMPTOMSTODAY']=pd.Series(x)
            # Hide axes
#             colors = plt.cm.BuPu(np.linspace(0, 0.5, 5))
            axtable.xaxis.set_visible(False) 
            axtable.yaxis.set_visible(False)
            axtable.axis('off')
            collabel=("TRIGGERSTODAY", "HELPMIGRAINETODAY", "SYMPTOMSTODAY")
            axtable.table(cellText=newdf.values,colLabels=collabel)
        else: 
            print('not in it')
    
    for i, row in dpd.loc[condition_actual, :].iterrows() : 
        for i in range(4):
            axs[i][0].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
            axs[i][0].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
            axs[i][0].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            axs[i][0].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
    for i, row in dpd.loc[condition_extra, :].iterrows() : 
        for i in range(4):
            axs[i][0].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

            axs[i][0].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
            axs[i][0].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            axs[i][0].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)

    
#     axs[0][0].plot(newdata10.index, newdata10['Temp'],label='Body Temperature')
    #                axs[1].plot(peaks.index,peaks['peaks_plot'],'#4DBD33')
    axs[0][0].plot(newdata1.index,5*newdata1['peaks_plot'],'#4DBD33')
    axs[0][0].plot(newdata10.index, newdata10['filtered_eda'],label='Phasic')
    axs[0][0].plot(newdata10.index,newdata10['tonic'],label='Tonic')
    axs[0][0].plot(df1.time,df1['tonic'],label='average tonic')
#     axs[2][0].plot(newdata.index, newdata['AccelX'],newdata.index, newdata['AccelY'],newdata.index, newdata['AccelZ'])
    axs[1][0].plot(data.index, (3*data['steps']),data.index, data['motion'])
    axs[2][0].plot(newdata_hrv.index, newdata_hrv['IBI'])
    axs[3][0].plot(newdata_hrv.index, newdata_hrv['HRV'])

    
    
#     plt.legend()
    axs[0][0].legend(fontsize='xx-small')
    axs[1][0].legend(fontsize='xx-small')
    axs[2][0].legend(fontsize='xx-small')
    axs[3][0].legend(fontsize='xx-small')
    
#     axs[0][0].set_ylabel('Temperature',fontsize='x-small')
#     axs[0].set_ylim (24 ,50)
    axs[0][0].set_ylabel('EDA',fontsize='x-small')
    data_min = min(newdata['EDA'])
    data_max =5
    axs[0][0].set_ylim(data_min ,data_max)
    axs[1][0].set_ylabel('Motion/Steps',fontsize='x-small')
    axs[2][0].set_ylabel('IBI',fontsize='x-small')
    axs[3][0].set_ylabel('HRV',fontsize='x-small')

    axs[3][0].set_xlabel('Time',fontsize='x-small')
    for ax in axs[0:,1]:
        
        ax.remove()
    
    # Hide x labels and tick labels for all but bottom plot.
    
    for ax in axs[0:,0]:
       ax.label_outer()
       hours = mdates.HourLocator(interval = 6)
       h_fmt = mdates.DateFormatter('%I:%M %p')
       ax.set_xlim([date+" 00:00:00-04:00", date+" 23:59:59-04:00"])
       #     ax.set_xlim(["00:00:00", "23:59:59"])
       xax = ax.get_xaxis()
       xax.set_major_locator(hours)
       xax.set_major_formatter(h_fmt)
       xax.set_minor_locator(mdates.HourLocator(interval = 1))
       xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
       xax.set_tick_params(which='major', pad=15)
    plt.tight_layout()
#     plt.show()
    plt.savefig('subplots_sensordata/binhourly/'+date+".png",bbox_inches="tight",dpi = 300)


C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


		computing motion...
		computing steps...
		computing stillness...
0
		computing features for time frame. Start index: 0 end index: -1


C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:181: RuntimeWarning: Mean of empty slice


2017-05-01


C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Chisel-Lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this

<IPython.core.display.Javascript object>

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


ValueError: view limit minimum -36826.64744502314 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

In [ ]:
#Hourly average ibi + EDA + Activity + (no temperature)
for f in range(0,len(files)):
    df = pd.read_csv(files[f], index_col=0, parse_dates=[0])
    df_hrv = pd.read_csv(files_hrv[f], index_col=0, parse_dates=[0])
    newdata_hrv = df_hrv.sort_index()
    newdata1 = df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')
    newdata1 = newdata1.sort_index()
    newdata_hrv=newdata_hrv.resample('1H').mean()
    newdata10=newdata1.resample('1H').mean()
    newdata=newdata1.resample('1T').mean()
    data=newdata1[['AccelX','AccelY','AccelZ']]
    features, steps, motion = computeAllAccelerometerFeatures(data, time_frames)
    data["steps"] = steps
    data["motion"] = motion
    condition_dpd = ( (dpd['MIGRAINESTART_TimeZone'].notnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )
#     newdata = newdata.drop_duplicates(subset=, keep='first')
    date=str(newdata.index[0].date())
    df_wthr=df_weather.loc[date]
    print(date)
#     fig, axs = plt.subplots(2, 9, sharex=0,gridspec_kw={'hspace': 0, 'wspace': 0})

    fig, axs = plt.subplots(ncols=2,nrows=4,figsize=(20, 10),gridspec_kw={'width_ratios' : [4,1]})
    gs = axs[1, 1].get_gridspec()
    
# remove the underlying axes

    axbig = fig.add_subplot(gs[0:2, 1])
    axtable = fig.add_subplot(gs[2:3, 1])
    objects = ('Sleep Duration', 'stress level', 'migraine index')
    x_pos = np.arange(len(objects))
#     fig.suptitle('Sensor Data Plots '+date) 
    y_pos = np.arange(len(objects))
    if date in dpd.index:
        row1=dpd.loc[date]

        if((len(row1)<2) & (len(row1)!=0) ):
            performance = [int(row1['SLEEPDURATIONHOURS']),int(row1['MIGRAINESEVERITY']),int(row1['SLEEPQUALITY'])]
            axbig.set_xticks(x_pos)
            axbig.set_xticklabels(('Sleep Duration','Migraine Severity','Stress Level'))
            axbig.bar(x_pos,performance, alpha=1,align='center')
            axbig.set_ylim(0,9)
            newdf=pd.DataFrame()
            for x in dpd.loc[date]["TRIGGERSTODAY"].values:
                newdf['TRIGGERSTODAY']=pd.Series(x)
            for x in dpd.loc[date]["HELPMIGRAINETODAY"].values:
                newdf['HELPMIGRAINETODAY']=pd.Series(x)
            for x in dpd.loc[date]["SYMPTOMSTODAY"].values:
                newdf['SYMPTOMSTODAY']=pd.Series(x)
            # Hide axes
#             colors = plt.cm.BuPu(np.linspace(0, 0.5, 5))
            axtable.xaxis.set_visible(False) 
            axtable.yaxis.set_visible(False)
            axtable.axis('off')
            collabel=("TRIGGERSTODAY", "HELPMIGRAINETODAY", "SYMPTOMSTODAY")
            axtable.table(cellText=newdf.values,colLabels=collabel)
        else: 
            print('not in it')
    
    for i, row in dpd.loc[condition_actual, :].iterrows() : 
        for i in range(4):
            axs[i][0].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
            axs[i][0].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
            axs[i][0].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            axs[i][0].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
    for i, row in dpd.loc[condition_extra, :].iterrows() : 
        for i in range(4):
            axs[i][0].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

            axs[i][0].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
            axs[i][0].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            axs[i][0].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)

    
#     axs[0][0].plot(newdata10.index, newdata10['Temp'],label='Body Temperature')
    #                axs[1].plot(peaks.index,peaks['peaks_plot'],'#4DBD33')
#     axs[0][0].plot(newdata1.index,5*newdata1['peaks_plot'],'#4DBD33')
    axs[0][0].plot(newdata10.index, newdata10['filtered_eda'],label='Phasic')
    axs[0][0].plot(newdata10.index,newdata10['tonic'],label='Tonic')
#     axs[2][0].plot(newdata.index, newdata['AccelX'],newdata.index, newdata['AccelY'],newdata.index, newdata['AccelZ'])
    axs[1][0].plot(data.index, (3*data['steps']),data.index, data['motion'])
    axs[2][0].plot(newdata_hrv.index, newdata_hrv['IBI'])
    axs[3][0].plot(newdata_hrv.index, newdata_hrv['HRV'])

    
    
#     plt.legend()
    axs[0][0].legend(fontsize='xx-small')
    axs[1][0].legend(fontsize='xx-small')
    axs[2][0].legend(fontsize='xx-small')
    axs[3][0].legend(fontsize='xx-small')
    
#     axs[0][0].set_ylabel('Temperature',fontsize='x-small')
#     axs[0].set_ylim (24 ,50)
    axs[0][0].set_ylabel('EDA',fontsize='x-small')
    data_min = min(newdata['EDA'])
    data_max =5
    axs[0][0].set_ylim(data_min ,data_max)
    axs[1][0].set_ylabel('Motion/Steps',fontsize='x-small')
    axs[2][0].set_ylabel('IBI',fontsize='x-small')
    axs[3][0].set_ylabel('HRV',fontsize='x-small')

    axs[3][0].set_xlabel('Time',fontsize='x-small')
    for ax in axs[0:,1]:
        
        ax.remove()
    
    # Hide x labels and tick labels for all but bottom plot.
    
    for ax in axs[0:,0]:
       ax.label_outer()
       hours = mdates.HourLocator(interval = 6)
       h_fmt = mdates.DateFormatter('%I:%M %p')
       ax.set_xlim([date+" 00:00:00-04:00", date+" 23:59:59-04:00"])
       #     ax.set_xlim(["00:00:00", "23:59:59"])
       xax = ax.get_xaxis()
       xax.set_major_locator(hours)
       xax.set_major_formatter(h_fmt)
       xax.set_minor_locator(mdates.HourLocator(interval = 1))
       xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
       xax.set_tick_params(which='major', pad=15)
    plt.tight_layout()
#     plt.show()
    plt.savefig('subplots_sensordata/binhourly/'+date+"-1.png",bbox_inches="tight",dpi = 300)


In [ ]:
#Hourly average
for f in range(0,len(files)):
    df = pd.read_csv(files[f], index_col=0, parse_dates=[0])
    df_hrv = pd.read_csv(files_hrv[f], index_col=0, parse_dates=[0])
    newdata_hrv = df_hrv.sort_index()
    newdata1 = df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')
    newdata1 = newdata1.sort_index()
#     newdata_hrv=newdata_hrv.resample('1T').mean()
    newdata10=newdata1.resample('1T').mean()
    newdata=newdata1.resample('1T').mean()
    data=newdata1[['AccelX','AccelY','AccelZ']]
    features, steps, motion = computeAllAccelerometerFeatures(data, time_frames)
    data["steps"] = steps
    data["motion"] = motion
    condition_dpd = ( (dpd['MIGRAINESTART_TimeZone'].notnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )
#     newdata = newdata.drop_duplicates(subset=, keep='first')
    date=str(newdata.index[0].date())
    df_wthr=df_weather.loc[date]
    print(date)
#     fig, axs = plt.subplots(2, 9, sharex=0,gridspec_kw={'hspace': 0, 'wspace': 0})

    fig, axs = plt.subplots(ncols=2,nrows=9,figsize=(20, 10),gridspec_kw={'width_ratios' : [4,1]})
    gs = axs[1, 1].get_gridspec()
    
# remove the underlying axes

    axbig = fig.add_subplot(gs[3:4, 1])
    axtable = fig.add_subplot(gs[5:7, 1])
    objects = ('Sleep Duration', 'stress level', 'migraine index')
    x_pos = np.arange(len(objects))
    fig.suptitle('Sensor Data Plots '+date) 
    y_pos = np.arange(len(objects))
    if date in dpd.index:
        row1=dpd.loc[date]

        if((len(row1)<2) & (len(row1)!=0) ):
            performance = [int(row1['SLEEPDURATIONHOURS']),int(row1['MIGRAINESEVERITY']),int(row1['SLEEPQUALITY'])]
            axbig.set_xticks(x_pos)
            axbig.set_xticklabels(('Sleep Duration','Migraine Severity','Stress Level'))
            axbig.bar(x_pos,performance, alpha=1,align='center')
            axbig.set_ylim(0,9)
            newdf=pd.DataFrame()
            for x in dpd.loc[date]["TRIGGERSTODAY"].values:
                newdf['TRIGGERSTODAY']=pd.Series(x)
            for x in dpd.loc[date]["HELPMIGRAINETODAY"].values:
                newdf['HELPMIGRAINETODAY']=pd.Series(x)
            for x in dpd.loc[date]["SYMPTOMSTODAY"].values:
                newdf['SYMPTOMSTODAY']=pd.Series(x)
            # Hide axes
#             colors = plt.cm.BuPu(np.linspace(0, 0.5, 5))
            axtable.xaxis.set_visible(False) 
            axtable.yaxis.set_visible(False)
            axtable.axis('off')
            collabel=("TRIGGERSTODAY", "HELPMIGRAINETODAY", "SYMPTOMSTODAY")
            axtable.table(cellText=newdf.values,colLabels=collabel)
        else: 
            print('not in it')
    
    for i, row in dpd.loc[condition_actual, :].iterrows() : 
        for i in range(9):
            axs[i][0].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
            axs[i][0].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
            axs[i][0].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            axs[i][0].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
    for i, row in dpd.loc[condition_extra, :].iterrows() : 
        for i in range(9):
            axs[i][0].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

            axs[i][0].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
            axs[i][0].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            axs[i][0].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)

    
    axs[0][0].plot(newdata10.index, newdata10['Temp'],label='Body Temperature')
    #                axs[1].plot(peaks.index,peaks['peaks_plot'],'#4DBD33')
    axs[1][0].plot(newdata1.index,5*newdata1['peaks_plot'],'#4DBD33')
    axs[1][0].plot(newdata.index, newdata['filtered_eda'],label='Phasic')
    axs[1][0].plot(newdata.index,newdata['tonic'],label='Tonic')
    axs[2][0].plot(newdata.index, newdata['AccelX'],newdata.index, newdata['AccelY'],newdata.index, newdata['AccelZ'])
    axs[3][0].plot(data.index, (3*data['steps']),data.index, data['motion'])
    axs[4][0].plot(newdata_hrv.index, newdata_hrv['IBI'])
    axs[5][0].plot(newdata_hrv.index, newdata_hrv['HRV'])
    axs[8][0].plot(df_wthr.index, df_wthr['apparentTemperature'],label='Atmospheric Temperature')
    axs[6][0].plot(df_wthr.index, df_wthr['humidity'],label='Humidity')
    axs[7][0].plot(df_wthr.index, df_wthr['pressure'],label='Pressure')
    axs[6][0].plot(df_wthr.index, df_wthr['cloudCover'],label='Cloud Cover')
    delta_t=axs[8][0].twinx()
    delta_t.plot(df_wthr.index, df_wthr['deltaTemp_1'],'c-',label='Delta Temperature')
    delta_h_cc=axs[6][0].twinx()
    delta_h_cc.plot(df_wthr.index, df_wthr['deltahumidity_1'],'g-',label='Delta Humidity')
    
    delta_h_cc.plot(df_wthr.index, df_wthr['deltacloudCover_1'],'m-',label='Delta Cloud Cover')
    delta_p=axs[7][0].twinx()
    delta_p.plot(df_wthr.index, df_wthr['delta_pressure'],'r-',label='Delta Pressure')
    delta_h_cc.set_ylabel('delta H & CC',color='g',fontsize='x-small')
    delta_p.set_ylabel('delta Pressure',color='r',fontsize='x-small')
    delta_t.set_ylabel('delta Temp',color='c',fontsize='x-small')
    
#     plt.legend()
    axs[0][0].legend(fontsize='xx-small')
    axs[1][0].legend(fontsize='xx-small')
    axs[2][0].legend(fontsize='xx-small')
    axs[3][0].legend(fontsize='xx-small')
    axs[4][0].legend(fontsize='xx-small')
    axs[5][0].legend(fontsize='xx-small')
    axs[6][0].legend(fontsize='xx-small')
    axs[7][0].legend(fontsize='xx-small')
    #            plt.xticks( rotation=25 )
    axs[0][0].set_ylabel('Temperature',fontsize='x-small')
#     axs[0].set_ylim (24 ,50)
    axs[1][0].set_ylabel('EDA',fontsize='x-small')
    data_min = min(newdata['EDA'])
    data_max =5
    axs[1][0].set_ylim(data_min ,data_max)
    axs[2][0].set_ylabel('Accelerometer',fontsize='x-small')
    axs[3][0].set_ylabel('Motion/Steps',fontsize='x-small')
    axs[4][0].set_ylabel('IBI',fontsize='x-small')
    axs[5][0].set_ylabel('HRV',fontsize='x-small')
    axs[8][0].set_ylabel('Temperature',fontsize='x-small')
    axs[6][0].set_ylabel('Hum & CC',fontsize='x-small')
    axs[7][0].set_ylabel('Pressure',fontsize='x-small')
#     axs[8].set_ylabel('CloudCover')
    axs[8][0].set_xlabel('Time',fontsize='x-small')
    for ax in axs[0:,1]:
        
        ax.remove()
    
    # Hide x labels and tick labels for all but bottom plot.
    
    for ax in axs[0:,0]:
       ax.label_outer()
       hours = mdates.HourLocator(interval = 6)
       h_fmt = mdates.DateFormatter('%I:%M %p')
       ax.set_xlim([date+" 00:00:00-04:00", date+" 23:59:59-04:00"])
       #     ax.set_xlim(["00:00:00", "23:59:59"])
       xax = ax.get_xaxis()
       xax.set_major_locator(hours)
       xax.set_major_formatter(h_fmt)
       xax.set_minor_locator(mdates.HourLocator(interval = 1))
       xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
       xax.set_tick_params(which='major', pad=15)
    plt.tight_layout()
#     plt.show()
    plt.savefig('subplots_sensordata/test3/'+date+".png",bbox_inches="tight",dpi = 300)


In [ ]:
#weather + stress leverl addition
for f in range(0,len(files)):
    df = pd.read_csv(files[f], index_col=0, parse_dates=[0])
    df_hrv = pd.read_csv(files_hrv[f], index_col=0, parse_dates=[0])
    newdata_hrv = df_hrv.sort_index()
    newdata1 = df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')
    newdata1 = newdata1.sort_index()
    newdata10=newdata1.resample('10t').mean()
    newdata=newdata1.resample('1T').mean()
    data=newdata1[['AccelX','AccelY','AccelZ']]
    features, steps, motion = computeAllAccelerometerFeatures(data, time_frames)
    data["steps"] = steps
    data["motion"] = motion
    condition_dpd = ( (dpd['MIGRAINESTART_TimeZone'].notnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )
#     newdata = newdata.drop_duplicates(subset=, keep='first')
    date=str(newdata.index[0].date())
    df_wthr=df_weather.loc[date]
    print(date)
#     fig, axs = plt.subplots(2, 9, sharex=0,gridspec_kw={'hspace': 0, 'wspace': 0})

    fig, axs = plt.subplots(ncols=2,nrows=9,figsize=(20, 10),gridspec_kw={'width_ratios' : [4,1]})
    gs = axs[1, 1].get_gridspec()
    
# remove the underlying axes

    axbig = fig.add_subplot(gs[3:4, 1])
    axtable = fig.add_subplot(gs[5:7, 1])
    objects = ('Sleep Duration', 'stress level', 'migraine index')
    y_pos = np.arange(len(objects))
    fig.suptitle('Sensor Data Plots '+date) 
    y_pos = np.arange(len(objects))
    if date in dpd.index:
        row1=dpd.loc[date]

        if((len(row1)<2) & (len(row1)!=0) ):
            performance = [int(row1['SLEEPDURATIONHOURS']),int(row1['MIGRAINESEVERITY']),int(row1['SLEEPQUALITY'])]
            axbig.set_xticks(x_pos)
            axbig.set_xticklabels(('Sleep Duration', 'Stress Level', 'Migraine'))
            axbig.bar(x_pos,performance, alpha=1,align='center',colors = 'rgb')
            newdf=pd.DataFrame()
            for x in dpd.loc[date]["TRIGGERSTODAY"].values:
                newdf['TRIGGERSTODAY']=pd.Series(x)
            for x in dpd.loc[date]["HELPMIGRAINETODAY"].values:
                newdf['HELPMIGRAINETODAY']=pd.Series(x)
            for x in dpd.loc[date]["SYMPTOMSTODAY"].values:
                newdf['SYMPTOMSTODAY']=pd.Series(x)
            # Hide axes
            colors = plt.cm.BuPu(np.linspace(0, 0.5, len(5)))
            axtable.xaxis.set_visible(False) 
            axtable.yaxis.set_visible(False)
            axtable.axis('off')
            collabel=("TRIGGERSTODAY", "HELPMIGRAINETODAY", "SYMPTOMSTODAY")
            axtable.table(cellText=newdf.values,colLabels=collabel)
        else: 
            print('not in it')
    
    for i, row in dpd.loc[condition_actual, :].iterrows() : 
        for i in range(9):
            axs[i][0].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
            axs[i][0].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
            axs[i][0].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            axs[i][0].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
    for i, row in dpd.loc[condition_extra, :].iterrows() : 
        for i in range(9):
            axs[i][0].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

            axs[i][0].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
            axs[i][0].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            axs[i][0].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)

    
    axs[0][0].plot(newdata10.index, newdata10['Temp'],label='Body Temperature')
    #                axs[1].plot(peaks.index,peaks['peaks_plot'],'#4DBD33')
    axs[1][0].plot(newdata1.index,5*newdata1['peaks_plot'],'#4DBD33')
    axs[1][0].plot(newdata.index, newdata['filtered_eda'],label='Phasic')
    axs[1][0].plot(newdata.index,newdata['tonic'],label='Tonic')
    axs[2][0].plot(newdata.index, newdata['AccelX'],newdata.index, newdata['AccelY'],newdata.index, newdata['AccelZ'])
    axs[3][0].plot(data.index, (3*data['steps']),data.index, data['motion'])
    axs[4][0].plot(newdata_hrv.index, newdata_hrv['IBI'])
    axs[5][0].plot(newdata_hrv.index, newdata_hrv['HRV'])
    axs[8][0].plot(df_wthr.index, df_wthr['apparentTemperature'],label='Atmospheric Temperature')
    axs[6][0].plot(df_wthr.index, df_wthr['humidity'],label='Humidity')
    axs[7][0].plot(df_wthr.index, df_wthr['pressure'],label='Pressure')
    axs[6][0].plot(df_wthr.index, df_wthr['cloudCover'],label='Cloud Cover')
    delta_t=axs[8][0].twinx()
    delta_t.plot(df_wthr.index, df_wthr['deltaTemp_1'],'c-',label='Delta Temperature')
    delta_h_cc=axs[6][0].twinx()
    delta_h_cc.plot(df_wthr.index, df_wthr['deltahumidity_1'],'g-',label='Delta Humidity')
    
    delta_h_cc.plot(df_wthr.index, df_wthr['deltacloudCover_1'],'m-',label='Delta Cloud Cover')
    delta_p=axs[7][0].twinx()
    delta_p.plot(df_wthr.index, df_wthr['delta_pressure'],'r-',label='Delta Pressure')
    delta_h_cc.set_ylabel('delta H & CC',color='g',fontsize='x-small')
    delta_p.set_ylabel('delta Pressure',color='r',fontsize='x-small')
    delta_t.set_ylabel('delta Temp',color='c',fontsize='x-small')
    
#     plt.legend()
    axs[0][0].legend(fontsize='xx-small')
    axs[1][0].legend(fontsize='xx-small')
    axs[2][0].legend(fontsize='xx-small')
    axs[3][0].legend(fontsize='xx-small')
    axs[4][0].legend(fontsize='xx-small')
    axs[5][0].legend(fontsize='xx-small')
    axs[6][0].legend(fontsize='xx-small')
    axs[7][0].legend(fontsize='xx-small')
    #            plt.xticks( rotation=25 )
    axs[0][0].set_ylabel('Temperature',fontsize='x-small')
#     axs[0].set_ylim (24 ,50)
    axs[1][0].set_ylabel('EDA',fontsize='x-small')
    data_min = min(newdata['EDA'])
    data_max =5
    axs[1][0].set_ylim(data_min ,data_max)
    axs[2][0].set_ylabel('Accelerometer',fontsize='x-small')
    axs[3][0].set_ylabel('Motion/Steps',fontsize='x-small')
    axs[4][0].set_ylabel('IBI',fontsize='x-small')
    axs[5][0].set_ylabel('HRV',fontsize='x-small')
    axs[8][0].set_ylabel('Temperature',fontsize='x-small')
    axs[6][0].set_ylabel('Hum & CC',fontsize='x-small')
    axs[7][0].set_ylabel('Pressure',fontsize='x-small')
#     axs[8].set_ylabel('CloudCover')
    axs[8][0].set_xlabel('Time',fontsize='x-small')
    for ax in axs[0:,1]:
        
        ax.remove()
    
    # Hide x labels and tick labels for all but bottom plot.
    
    for ax in axs[0:,0]:
       ax.label_outer()
       hours = mdates.HourLocator(interval = 6)
       h_fmt = mdates.DateFormatter('%I:%M %p')
       ax.set_xlim([date+" 00:00:00-04:00", date+" 23:59:59-04:00"])
       #     ax.set_xlim(["00:00:00", "23:59:59"])
       xax = ax.get_xaxis()
       xax.set_major_locator(hours)
       xax.set_major_formatter(h_fmt)
       xax.set_minor_locator(mdates.HourLocator(interval = 1))
       xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
       xax.set_tick_params(which='major', pad=15)
    plt.tight_layout()
#     plt.show()
    plt.savefig('subplots_sensordata/test4/'+date+".png",bbox_inches="tight",dpi = 300)


In [ ]:
# fig, axs = plt.subplots(2, 9)
fig, axs = plt.subplots(ncols=2, nrows=9,figsize=(20, 10))
gs = axs[1, 1].get_gridspec()
# remove the underlying axes

axbig = fig.add_subplot(gs[3:4, 1])
axbig = fig.add_subplot(gs[3:4, 1])
objects = ('Sleep Duration', 'stress level', 'migraine index')
y_pos = np.arange(len(objects))
performance = [int(dpd['SLEEPDURATIONHOURS'][3]),int(dpd['MIGRAINESEVERITY'][3]),int(dpd['SLEEPQUALITY'][3])]
axbig.bar(y_pos,performance, alpha=1,align='center')
axbig.set_xticks(y_pos, objects)
fig.suptitle('Sensor Data Plots '+date) 
y_pos = np.arange(len(objects))
if date in dpd.index:
    row=dpd.loc[date]
    performance = [int(row['SLEEPDURATIONHOURS']),int(row['MIGRAINESEVERITY']),int(row['SLEEPQUALITY'])]
    axbig.bar(y_pos,performance, alpha=1,align='center')
    axbig.set_xticks(y_pos, objects)
for i, row in dpd.loc[condition_dpd, :].iterrows() : 


    if (date in date_extra): 
        #extrapolated date time
#         print('extrapolated date time')
        for ax in axs[0:,0]:
            ax.axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
            ax.axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
            ax.axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            ax.axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
    else:
#             print('real date time')
        for ax in axs[0:,0]:
            ax.axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

            ax.axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
            ax.axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            ax.axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)

axs[0][0].plot(newdata10.index, newdata10['Temp'],label='Body Temperature')
#                axs[1].plot(peaks.index,peaks['peaks_plot'],'#4DBD33')
axs[1][0].plot(newdata1.index,5*newdata1['peaks_plot'],'#4DBD33')
axs[1][0].plot(newdata.index, newdata['filtered_eda'],label='Phasic')
axs[1][0].plot(newdata.index,newdata['tonic'],label='Tonic')
axs[2][0].plot(newdata.index, newdata['AccelX'],newdata.index, newdata['AccelY'],newdata.index, newdata['AccelZ'])
axs[3][0].plot(data.index, (3*data['steps']),data.index, data['motion'])
axs[4][0].plot(newdata_hrv.index, newdata_hrv['IBI'])
axs[5][0].plot(newdata_hrv.index, newdata_hrv['HRV'])
axs[8][0].plot(df_wthr.index, df_wthr['apparentTemperature'],label='Atmospheric Temperature')
axs[6][0].plot(df_wthr.index, df_wthr['humidity'],label='Humidity')
axs[7][0].plot(df_wthr.index, df_wthr['pressure'],label='Pressure')
axs[6][0].plot(df_wthr.index, df_wthr['cloudCover'],label='Cloud Cover')


#     plt.legend()
axs[0][0].legend(fontsize='xx-small')
axs[1][0].legend(fontsize='xx-small')
axs[2][0].legend(fontsize='xx-small')
axs[3][0].legend(fontsize='xx-small')
axs[4][0].legend(fontsize='xx-small')
axs[5][0].legend(fontsize='xx-small')
axs[6][0].legend(fontsize='xx-small')
axs[7][0].legend(fontsize='xx-small')
#            plt.xticks( rotation=25 )
axs[0][0].set_ylabel('Temperature',fontsize='x-small')
#     axs[0].set_ylim (24 ,50)
axs[1][0].set_ylabel('EDA',fontsize='x-small')
data_min = min(newdata['EDA'])
data_max =5
axs[1][0].set_ylim(data_min ,data_max)
axs[2][0].set_ylabel('Accelerometer',fontsize='x-small')
axs[3][0].set_ylabel('Motion/Steps',fontsize='x-small')
axs[4][0].set_ylabel('IBI',fontsize='x-small')
axs[5][0].set_ylabel('HRV',fontsize='x-small')
axs[8][0].set_ylabel('Temperature',fontsize='x-small')
axs[6][0].set_ylabel('Hum & CC',fontsize='x-small')
axs[7][0].set_ylabel('Pressure',fontsize='x-small')
#     axs[8].set_ylabel('CloudCover')
axs[8][0].set_xlabel('Time',fontsize='x-small')
for ax in axs[0:,1]:

    ax.remove()
for ax in axs[0:,0]:

    ax.set_xticklabels([])

# Hide x labels and tick labels for all but bottom plot.
for ax in axs[0:,0]:
   ax.label_outer()
   hours = mdates.HourLocator(interval = 6)
   h_fmt = mdates.DateFormatter('%I:%M %p')
   ax.set_xlim([date+" 00:00:00-04:00", date+" 23:59:59-04:00"])
   #     ax.set_xlim(["00:00:00", "23:59:59"])
   xax = ax.get_xaxis()
   xax.set_major_locator(hours)
   xax.set_major_formatter(h_fmt)
   xax.set_minor_locator(mdates.HourLocator(interval = 1))
   xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
   xax.set_tick_params(which='major', pad=15)
plt.savefig('subplots_sensordata/test/'+date+".png",bbox_inches="tight",dpi = 300)
#     plt.close()
#     axs1.savefig('subplots_sensordata/ '+date+"-da3.png",bbox_inches="tight",dpi = 900)


In [ ]:
df_weather

In [ ]:
df_weather['Gradienthumidity_12'] = np.gradient(df_weather['humidity'].rolling(center=False,window=12).mean())

In [ ]:
df_weather['GradientcloudCover_12'] = np.gradient(df_weather['cloudCover'].rolling(center=False,window=12).mean())

In [ ]:
df_weather['GradientTemp_12'] = np.gradient(df_weather['apparentTemperature'].rolling(center=False,window=12).mean())

In [ ]:
df_weather['Gradienthumidity_24'] = np.gradient(df_weather['humidity'].rolling(center=False,window=24).mean())
df_weather['GradientcloudCover_24'] = np.gradient(df_weather['cloudCover'].rolling(center=False,window=24).mean())
df_weather['GradientTemp_24'] = np.gradient(df_weather['apparentTemperature'].rolling(center=False,window=24).mean())

In [ ]:
df_weather['Gradienthumidity_6'] = np.gradient(df_weather['humidity'].rolling(center=False,window=6).mean())
df_weather['GradientcloudCover_6'] = np.gradient(df_weather['cloudCover'].rolling(center=False,window=6).mean())
# df_weather['GradientTemp_1'] = np.gradient(df_weather['apparentTemperature'])


In [ ]:
df_weather['deltaTemp_1'] = df_weather.apparentTemperature.diff().fillna(0).astype(float)
df_weather['deltahumidity_1'] = df_weather.humidity.diff().fillna(0).astype(float)
df_weather['deltacloudCover_1'] = df_weather.cloudCover.diff().fillna(0).astype(float)
df_weather['delta_pressure'] = df_weather.pressure.diff().fillna(0).astype(float)

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

# Base time series
base_t = pd.date_range("2017-07-20 11:00", "2017-07-20 12:00", freq="T")

# We add some random noise to achieve non uniform spacing
t = base_t + pd.to_timedelta(5 * np.random.randn(len(base_t)), unit='s')
assert t.is_monotonic_increasing

# Extract numerical values
values = (t - t[0]).total_seconds()

# My function
y = pd.Series(np.cos(0.1 * values))

# Derivative! Requires NumPy >= 1.13
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.gradient.html#numpy.gradient
dy = np.gradient(y, values)

# Plotting
fig, ax = plt.subplots(2, sharex=True, figsize=(6, 6))

ax[0].plot(t, y)
ax[1].plot(t, dy, color="C1")

In [ ]:
df_weather.loc['2017-04-17']['GradientTemp']

In [ ]:
df_weather.loc['2017-04-17']['apparentTemperature']

In [ ]:
df_weather['GradientTemp_1'] = df_weather.apparentTemperature.diff().fillna(0).astype(float)
df_weather.loc['2017-04-17']['GradientTemp_1']

df_weather['GradientTemp_1'] =df_weather.apparentTemperature.rolling('3601s').apply(lambda x: x[-1] - x[0]) / 2

In [ ]:
df_weather['GradientTemp_1']

In [ ]:
objects = ('Sleep', 'stress', 'migraine')
x_pos = np.arange(3)
for i, row in dpd.loc[condition_dpd, :].iterrows() :     
    fig, axs = plt.subplots(2,figsize=(20, 10))
    performance = [int(row['SLEEPDURATIONHOURS']),int(row['MIGRAINESEVERITY']),int(row['SLEEPQUALITY'])]
    # dpd['MIGRAINESEVERITY'][0] 
    
    plt.bar(x_pos,performance, alpha=1,align='center')
#     ax.set_yticks(x_pos)
    plt.xticks(x_pos, ('Sleep Duration', 'Stress Level', 'Migraine'))
#     ax.set_yticklabels(objects) 
#     axs.set_yticks(y_pos, objects)
    
#     axs.set_ylabel('Usage')
#     axs.set_title('Programming language usage')

In [ ]:
date='2017-04-21'

In [ ]:
'2017-04-21' in dpd.index
# dpd.loc['2017-04-05']

In [ ]:
cnd=(row1['SLEEPDURATIONHOURS'].notnull() & row1['MIGRAINESEVERITY'].notnull() & row1['SLEEPQUALITY'].notnull())

In [ ]:
dpd['SLEEPDURATIONHOURS'] = dpd['SLEEPDURATIONHOURS'].fillna(0)
dpd['MIGRAINESEVERITY'] = dpd['MIGRAINESEVERITY'].fillna(0)
dpd['SLEEPQUALITY'] = dpd['SLEEPQUALITY'].fillna(0)

In [ ]:

fig = plt.figure(figsize=(20, 7))
grid = plt.GridSpec(8, 4, wspace=0.2, hspace=0.2)
left_bar=plt.subplot(grid[3, 0])
x0=plt.subplot(grid[0, 1:3])
x1=plt.subplot(grid[1, 1:3])
x2=plt.subplot(grid[2, 1:3])
x3=plt.subplot(grid[3, 1:3])
x4=plt.subplot(grid[4, 1:3])
x5=plt.subplot(grid[5, 1:3])
x6=plt.subplot(grid[6, 1:3])
x7=plt.subplot(grid[7, 1:3])
right_bar=plt.subplot(grid[3, 3])
#testing weather data and derivatives

#     right_bar.plot(x, y, 'ok', markersize=3, alpha=0.2)
df = pd.read_csv('eda-explorer-master/test/2017-04-28.csv', index_col=0, parse_dates=[0])
date=str(df.index[0].date())
df_wthr=df_weather.loc[date]
fig.suptitle('Sensor Data Plots '+date) 
objects = ('Sleep', 'stress', 'migraine')
y_pos = np.arange(len(objects))
x_pos = np.arange(3)

In [ ]:
fig, ax = plt.subplots()

newdf=pd.DataFrame()
for x in dpd.loc[date]["TRIGGERSTODAY"].values:
    newdf['TRIGGERSTODAY']=pd.Series(x)
for x in dpd.loc[date]["HELPMIGRAINETODAY"].values:
    newdf['HELPMIGRAINETODAY']=pd.Series(x)
for x in dpd.loc[date]["SYMPTOMSTODAY"].values:
    newdf['SYMPTOMSTODAY']=pd.Series(x)
# Hide axes
ax.xaxis.set_visible(False) 
ax.yaxis.set_visible(False)
ax.axis('off')
# Table from Ed Smith answer
# clust_data=dpd.loc[date][["HELPMIGRAINETODAY", "SYMPTOMSTODAY", "TRIGGERSTODAY"]].values
# clust_data = np.random.random((1,3))
collabel=("HELPMIGRAINETODAY", "SYMPTOMSTODAY", "TRIGGERSTODAY")
ax.table(cellText=newdf.values,colLabels=collabel)

In [ ]:
clust_data

In [ ]:
dpd.loc[date][["HELPMIGRAINETODAY", "SYMPTOMSTODAY", "TRIGGERSTODAY"]].values

In [ ]:
newdf=pd.DataFrame()
for x in dpd.loc[date]["TRIGGERSTODAY"].values:
    newdf['TRIGGERSTODAY']=pd.Series(x)
for x in dpd.loc[date]["HELPMIGRAINETODAY"].values:
    newdf['HELPMIGRAINETODAY']=pd.Series(x)
for x in dpd.loc[date]["SYMPTOMSTODAY"].values:
    newdf['SYMPTOMSTODAY']=pd.Series(x)



In [ ]:
newdf.head()

In [ ]:
dpd.loc[date][["HELPMIGRAINETODAY", "SYMPTOMSTODAY", "TRIGGERSTODAY"]]

In [ ]:
date='2017-04-28'
import seaborn as sns
fig = plt.figure(figsize=(20, 7))
grid = plt.GridSpec(8, 4, wspace=0.2, hspace=0.2)
left_bar=plt.subplot(grid[3, 0])
x0 = fig.add_subplot(grid[0, 1:3])
x1 = fig.add_subplot(grid[1, 1:3])
x2 = fig.add_subplot(grid[2, 1:3])
x3 = fig.add_subplot(grid[3, 1:3])
x4 = fig.add_subplot(grid[4, 1:3])
x5 = fig.add_subplot(grid[5, 1:3])
x6 = fig.add_subplot(grid[6, 1:3])
x7 = fig.add_subplot(grid[7, 1:3])
right_bar = fig.add_subplot(grid[3, 3])
if date in dpd.index:
    row1=dpd.loc[date]

    if(len(row1)<2):
        performance = [int(row1['SLEEPDURATIONHOURS']),int(row1['MIGRAINESEVERITY']),int(row1['SLEEPQUALITY'])]
        right_bar.set_xticks(x_pos)
        right_bar.set_xticklabels(('Sleep Duration', 'Stress Level', 'Migraine'))
        right_bar.bar(x_pos,performance, alpha=1,align='center')
        newdf=pd.DataFrame()
        for x in dpd.loc[date]["TRIGGERSTODAY"].values:
            newdf['TRIGGERSTODAY']=pd.Series(x)
        for x in dpd.loc[date]["HELPMIGRAINETODAY"].values:
            newdf['HELPMIGRAINETODAY']=pd.Series(x)
        for x in dpd.loc[date]["SYMPTOMSTODAY"].values:
            newdf['SYMPTOMSTODAY']=pd.Series(x)
        # Hide axes
        left_bar.xaxis.set_visible(False) 
        left_bar.yaxis.set_visible(False)
        left_bar.axis('off')
        collabel=("HELPMIGRAINETODAY", "SYMPTOMSTODAY", "TRIGGERSTODAY")
        left_bar.table(cellText=newdf.values,colLabels=collabel)
    else: 
        print('not in it')
if(len(dpd.loc[date])<2):  
    row=dpd.loc[date]
    for i in range(8):
        plt.subplot(grid[i, 1:3]).axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
        #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
        plt.subplot(grid[i, 1:3]).axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
        plt.subplot(grid[i, 1:3]).axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
        plt.subplot(grid[i, 1:3]).axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)

x0.plot(df_wthr.index, df_wthr['apparentTemperature'],label='Atmospheric Temperature')
x1.plot(df_wthr.index, df_wthr['humidity'],label='Humidity')
x2.plot(df_wthr.index, df_wthr['pressure'],label='Pressure')
x3.plot(df_wthr.index, df_wthr['cloudCover'],label='Cloud Cover')
x4.plot(df_wthr.index, df_wthr['deltaTemp_1'],label='Delta Temperature')
x5.plot(df_wthr.index, df_wthr['deltahumidity_1'],label='Delta Humidity')
x6.plot(df_wthr.index, df_wthr['delta_pressure'],label='Delta Pressure')

x7.plot(df_wthr.index, df_wthr['deltacloudCover_1'],label='Delta Cloud Cover')
# plt.setp(x7.get_xticklabels(), visible=False)
x0.legend(fontsize='xx-small')
x1.legend(fontsize='xx-small')
x2.legend(fontsize='xx-small')
x3.legend(fontsize='xx-small')
x4.legend(fontsize='xx-small')

x5.legend(fontsize='xx-small')
x6.legend(fontsize='xx-small')
x7.legend(fontsize='xx-small')
x7.set_xlabel('Time',fontsize='x-small')
plt.savefig('subplots_sensordata/weatherdataplots/delta_bar/'+date+".png",bbox_inches="tight",dpi = 300)


In [ ]:
fig = plt.figure(figsize=(20, 7))
grid = plt.GridSpec(8, 4, wspace=0.2, hspace=0.2)
left_bar=plt.subplot(grid[3, 0])
x0 = fig.add_subplot(grid[0, 1:3])
# x0=plt.subplot(grid[0, 1:3])
x1=plt.subplot(grid[1, 1:3])
x2=plt.subplot(grid[2, 1:3])
x3=plt.subplot(grid[3, 1:3])
x4=plt.subplot(grid[4, 1:3])
x5=plt.subplot(grid[5, 1:3])
x6=plt.subplot(grid[6, 1:3])
x7=plt.subplot(grid[7, 1:3])
right_bar=plt.subplot(grid[3, 3])
if date in dpd.index:
    row1=dpd.loc[date]

    if(len(row1)<2):
        performance = [int(row1['SLEEPDURATIONHOURS']),int(row1['MIGRAINESEVERITY']),int(row1['SLEEPQUALITY'])]

        plt.subplot(grid[3, 3]).bar(x_pos,performance, alpha=1,align='center')
        #     ax.set_yticks(x_pos)
        plt.subplot(grid[3, 3]).set_xticks(x_pos, ('Sleep Duration', 'Stress Level', 'Migraine'))
        left_bar.bar(x_pos,performance, alpha=1,align='center')
        #     ax.set_yticks(x_pos)
        left_bar.set_xticks(x_pos, ('Sleep Duration', 'Stress Level', 'Migraine'))
    else: 
        print('not in it')
if(len(dpd.loc[date])<2):  
    row=dpd.loc[date]
    for i in range(8):
        plt.subplot(grid[i, 1:3]).axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
        #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
        plt.subplot(grid[i, 1:3]).axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
        plt.subplot(grid[i, 1:3]).axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
        plt.subplot(grid[i, 1:3]).axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
#         else:
#             print('real date time')
#             for i in range(8):
#                 axs[i].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
#                 #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

#                 axs[i].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
#                 axs[i].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
#                 axs[i].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)

x0.plot(df_wthr.index, df_wthr['apparentTemperature'],label='Atmospheric Temperature')
x1.plot(df_wthr.index, df_wthr['humidity'],label='Humidity')
x2.plot(df_wthr.index, df_wthr['pressure'],label='Pressure')
x3.plot(df_wthr.index, df_wthr['cloudCover'],label='Cloud Cover')
x4.plot(df_wthr.index, df_wthr['deltaTemp_1'],label='Delta Temperature')
x5.plot(df_wthr.index, df_wthr['deltahumidity_1'],label='Delta Humidity')
x6.plot(df_wthr.index, df_wthr['delta_pressure'],label='Delta Pressure')

x7.plot(df_wthr.index, df_wthr['deltacloudCover_1'],label='Delta Cloud Cover')

#     axs[8].plot(df_wthr.index, df_wthr['GradientcloudCover_24'],label='Delta Cloud Cover 24')
#     axs[7].plot(df_wthr.index, df_wthr['GradientcloudCover'],label='Delta Cloud Cover')


#     plt.legend()
x0.legend(fontsize='xx-small')
x1.legend(fontsize='xx-small')
x2.legend(fontsize='xx-small')
x3.legend(fontsize='xx-small')
x4.legend(fontsize='xx-small')
x5.legend(fontsize='xx-small')
x6.legend(fontsize='xx-small')
x7.legend(fontsize='xx-small')
#            plt.xticks( rotation=25 )
#     axs[0].set_ylabel('Temperature',fontsize='x-small')
# #     axs[0].set_ylim(24 ,50)
#     axs[1].set_ylabel('EDA',fontsize='x-small')
#     data_min = min(newdata['EDA'])
#     data_max =5
#     axs[1].set_ylim(data_min ,data_max)
#     axs[2].set_ylabel('Accelerometer',fontsize='x-small')
#     axs[3].set_ylabel('Motion/Steps',fontsize='x-small')
#     axs[4].set_ylabel('IBI',fontsize='x-small')
#     axs[5].set_ylabel('HRV',fontsize='x-small')
#     axs[8].set_ylabel('Temperature',fontsize='x-small')
#     axs[6].set_ylabel('Hum & CC',fontsize='x-small')
#     axs[7].set_ylabel('Pressure',fontsize='x-small')
# #     axs[8].set_ylabel('CloudCover')
x0.set_xlabel('Time',fontsize='x-small')

#        xax.set_tick_params(which='major', pad=15)
# Hide x labels and tick labels for all but bottom plot.
# for i in range(8):
   
#    hours = mdates.HourLocator(interval = 6)
#    h_fmt = mdates.DateFormatter('%I:%M %p')
# #        ax.xaxis.set_minor_locator(minutes)
# #        ax.xaxis.set_major_locator(hours)
# #        ax.xaxis.set_major_formatter(h_fmt)
#    # set date ticks to something sensible:
#    ax=plt.subplot(grid[i, 1:3])
#    ax.label_outer()
#    
#    #     ax.set_xlim(["00:00:00", "23:59:59"])
#    xax = ax.get_xaxis()
#    xax.set_major_locator(hours)
#    xax.set_major_formatter(h_fmt)
#    xax.set_minor_locator(mdates.HourLocator(interval = 1))
#    xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
# #        xax.set_tick_params(which='major', pad=15)
plt.savefig('subplots_sensordata/weatherdataplots/delta_bar/'+date+".png",bbox_inches="tight",dpi = 300)


In [ ]:
#testing weather data and derivatives
for f in range(0,len(files)):
    fig = plt.figure(figsize=(20, 7))
    grid = plt.GridSpec(8, 4, wspace=0.2, hspace=0.2)
    left_bar=plt.subplot(grid[3, 0])
    x0=plt.subplot(grid[0, 1:3])
    x1=plt.subplot(grid[1, 1:3])
    x2=plt.subplot(grid[2, 1:3])
    x3=plt.subplot(grid[3, 1:3])
    x4=plt.subplot(grid[4, 1:3])
    x5=plt.subplot(grid[5, 1:3])
    x6=plt.subplot(grid[6, 1:3])
    x7=plt.subplot(grid[7, 1:3])
    right_bar=plt.subplot(grid[3, 3])
#     right_bar.plot(x, y, 'ok', markersize=3, alpha=0.2)
    df = pd.read_csv(files[f], index_col=0, parse_dates=[0])
    date=str(df.index[0].date())
    df_wthr=df_weather.loc[date]
    fig.suptitle('Sensor Data Plots '+date) 
    objects = ('Sleep', 'stress', 'migraine')
    y_pos = np.arange(len(objects))
    x_pos = np.arange(3)
    if date in dpd.index:
        
        row1=dpd.loc[date]    
        if(len(m)<2):
            performance = [int(row1['SLEEPDURATIONHOURS']),int(row1['MIGRAINESEVERITY']),int(row1['SLEEPQUALITY'])]

            left_bar.bar(x_pos,performance, alpha=1,align='center')
            #     ax.set_yticks(x_pos)
            left_bar.xticks(x_pos, ('Sleep Duration', 'Stress Level', 'Migraine'))
            right_bar.bar(x_pos,performance, alpha=1,align='center')
            #     ax.set_yticks(x_pos)
            right_bar.xticks(x_pos, ('Sleep Duration', 'Stress Level', 'Migraine'))
#     for i, row in dpd.loc[condition_dpd, :].iterrows() :         
# #         performance = [int(row['SLEEPDURATIONHOURS']),int(row['MIGRAINESEVERITY']),int(row['SLEEPQUALITY'])]
# #         axs[9].set_yticks(y_pos, objects)
# #         axs[9].ylabel('Usage')
# #         axs[9].title('Programming language usage')
#     #print(t)
# #         print(row['dt_diary'].date() + 'while date looping is '+date )
# #         if(date==str(row['dt_diary'].date())):
#         if (date in date_extra): 
#             #extrapolated date time
# #         print('extrapolated date time')
#             for i in range(8):
#                 axs[i].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
#                 #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
#                 axs[i].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
#                 axs[i].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
#                 axs[i].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
#         else:
# #             print('real date time')
#             for i in range(8):
#                 axs[i].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
#                 #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

#                 axs[i].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
#                 axs[i].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
#                 axs[i].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
    
    x0.plot(df_wthr.index, df_wthr['apparentTemperature'],label='Atmospheric Temperature')
    x1.plot(df_wthr.index, df_wthr['humidity'],label='Humidity')
    x2.plot(df_wthr.index, df_wthr['pressure'],label='Pressure')
    x3.plot(df_wthr.index, df_wthr['cloudCover'],label='Cloud Cover')
    x4.plot(df_wthr.index, df_wthr['deltaTemp_1'],label='Delta Temperature')
    x5.plot(df_wthr.index, df_wthr['deltahumidity_1'],label='Delta Humidity')
    x6.plot(df_wthr.index, df_wthr['delta_pressure'],label='Delta Pressure')
    x7.plot(df_wthr.index, df_wthr['deltacloudCover_1'],label='Delta Cloud Cover')
    
#     axs[8].plot(df_wthr.index, df_wthr['GradientcloudCover_24'],label='Delta Cloud Cover 24')
#     axs[7].plot(df_wthr.index, df_wthr['GradientcloudCover'],label='Delta Cloud Cover')
    
    
#     plt.legend()
    x0.legend(fontsize='xx-small')
    x1.legend(fontsize='xx-small')
    x2.legend(fontsize='xx-small')
    x3.legend(fontsize='xx-small')
    x4.legend(fontsize='xx-small')
    x5.legend(fontsize='xx-small')
    x6.legend(fontsize='xx-small')
    x7.legend(fontsize='xx-small')
    #            plt.xticks( rotation=25 )
#     axs[0].set_ylabel('Temperature',fontsize='x-small')
# #     axs[0].set_ylim(24 ,50)
#     axs[1].set_ylabel('EDA',fontsize='x-small')
#     data_min = min(newdata['EDA'])
#     data_max =5
#     axs[1].set_ylim(data_min ,data_max)
#     axs[2].set_ylabel('Accelerometer',fontsize='x-small')
#     axs[3].set_ylabel('Motion/Steps',fontsize='x-small')
#     axs[4].set_ylabel('IBI',fontsize='x-small')
#     axs[5].set_ylabel('HRV',fontsize='x-small')
#     axs[8].set_ylabel('Temperature',fontsize='x-small')
#     axs[6].set_ylabel('Hum & CC',fontsize='x-small')
#     axs[7].set_ylabel('Pressure',fontsize='x-small')
# #     axs[8].set_ylabel('CloudCover')
    x0.set_xlabel('Time',fontsize='x-small')

    # Hide x labels and tick labels for all but bottom plot.
#     for ax in axs:
#        ax.label_outer()
#        hours = mdates.HourLocator(interval = 6)
#        h_fmt = mdates.DateFormatter('%I:%M %p')
#     #        ax.xaxis.set_minor_locator(minutes)
#     #        ax.xaxis.set_major_locator(hours)
#     #        ax.xaxis.set_major_formatter(h_fmt)
#        # set date ticks to something sensible:
#        ax.set_xlim([date+" 00:00:00-04:00", date+" 23:59:59-04:00"])
#        #     ax.set_xlim(["00:00:00", "23:59:59"])
#        xax = ax.get_xaxis()
#        xax.set_major_locator(hours)
#        xax.set_major_formatter(h_fmt)
#        xax.set_minor_locator(mdates.HourLocator(interval = 1))
#        xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
#        xax.set_tick_params(which='major', pad=15)
    plt.savefig('subplots_sensordata/weatherdataplots/delta_bar/'+date+".png",bbox_inches="tight",dpi = 300)


In [ ]:
#testing weather data and derivatives
for f in range(0,len(files)):
#     io.StringIO(df.to_csv()), index_col=0
#     pd.read_csv(io.StringIO(f),index_col=0)
    df = pd.read_csv(files[f], index_col=0, parse_dates=[0])
    date=str(df.index[0].date())
    df_wthr=df_weather.loc[date]
    fig, axs = plt.subplots(9,figsize=(20, 10),sharex = False, sharey = False,gridspec_kw={'hspace': 0})
    fig.suptitle('Sensor Data Plots '+date) 
    objects = ('Sleep', 'stress', 'migraine')
    y_pos = np.arange(len(objects))
    x_pos = np.arange(3)
    if date in dpd.index:
        
        row1=dpd.loc[date]    
        if(len(row1)<2):
            performance = [int(row1['SLEEPDURATIONHOURS']),int(row1['MIGRAINESEVERITY']),int(row1['SLEEPQUALITY'])]
            # dpd['MIGRAINESEVERITY'][0] 
            plt.bar(x_pos,performance, alpha=1,align='center')
            #     ax.set_yticks(x_pos)
            plt.xticks(x_pos, ('Sleep Duration', 'Stress Level', 'Migraine'))
#     for i, row in dpd.loc[condition_dpd, :].iterrows() :         
# #         performance = [int(row['SLEEPDURATIONHOURS']),int(row['MIGRAINESEVERITY']),int(row['SLEEPQUALITY'])]
# #         axs[9].set_yticks(y_pos, objects)
# #         axs[9].ylabel('Usage')
# #         axs[9].title('Programming language usage')
#     #print(t)
# #         print(row['dt_diary'].date() + 'while date looping is '+date )
# #         if(date==str(row['dt_diary'].date())):
#         if (date in date_extra): 
#             #extrapolated date time
# #         print('extrapolated date time')
#             for i in range(8):
#                 axs[i].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
#                 #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
#                 axs[i].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
#                 axs[i].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
#                 axs[i].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
#         else:
# #             print('real date time')
#             for i in range(8):
#                 axs[i].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
#                 #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

#                 axs[i].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
#                 axs[i].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
#                 axs[i].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
    
    axs[0].plot(df_wthr.index, df_wthr['apparentTemperature'],label='Atmospheric Temperature')
    axs[1].plot(df_wthr.index, df_wthr['humidity'],label='Humidity')
    axs[2].plot(df_wthr.index, df_wthr['pressure'],label='Pressure')
    axs[3].plot(df_wthr.index, df_wthr['cloudCover'],label='Cloud Cover')
    axs[4].plot(df_wthr.index, df_wthr['deltaTemp_1'],label='Delta Temperature')
    axs[5].plot(df_wthr.index, df_wthr['deltahumidity_1'],label='Delta Humidity')
    axs[6].plot(df_wthr.index, df_wthr['delta_pressure'],label='Delta Pressure')
    axs[7].plot(df_wthr.index, df_wthr['deltacloudCover_1'],label='Delta Cloud Cover')
    
#     axs[8].plot(df_wthr.index, df_wthr['GradientcloudCover_24'],label='Delta Cloud Cover 24')
#     axs[7].plot(df_wthr.index, df_wthr['GradientcloudCover'],label='Delta Cloud Cover')
    
    
#     plt.legend()
    axs[0].legend(fontsize='xx-small')
    axs[1].legend(fontsize='xx-small')
    axs[2].legend(fontsize='xx-small')
    axs[3].legend(fontsize='xx-small')
    axs[4].legend(fontsize='xx-small')
    axs[5].legend(fontsize='xx-small')
    axs[6].legend(fontsize='xx-small')
    axs[7].legend(fontsize='xx-small')
    #            plt.xticks( rotation=25 )
#     axs[0].set_ylabel('Temperature',fontsize='x-small')
# #     axs[0].set_ylim(24 ,50)
#     axs[1].set_ylabel('EDA',fontsize='x-small')
#     data_min = min(newdata['EDA'])
#     data_max =5
#     axs[1].set_ylim(data_min ,data_max)
#     axs[2].set_ylabel('Accelerometer',fontsize='x-small')
#     axs[3].set_ylabel('Motion/Steps',fontsize='x-small')
#     axs[4].set_ylabel('IBI',fontsize='x-small')
#     axs[5].set_ylabel('HRV',fontsize='x-small')
#     axs[8].set_ylabel('Temperature',fontsize='x-small')
#     axs[6].set_ylabel('Hum & CC',fontsize='x-small')
#     axs[7].set_ylabel('Pressure',fontsize='x-small')
# #     axs[8].set_ylabel('CloudCover')
    axs[0].set_xlabel('Time',fontsize='x-small')

    # Hide x labels and tick labels for all but bottom plot.
    for ax in axs:
       ax.label_outer()
       hours = mdates.HourLocator(interval = 6)
       h_fmt = mdates.DateFormatter('%I:%M %p')
    #        ax.xaxis.set_minor_locator(minutes)
    #        ax.xaxis.set_major_locator(hours)
    #        ax.xaxis.set_major_formatter(h_fmt)
       # set date ticks to something sensible:
       ax.set_xlim([date+" 00:00:00-04:00", date+" 23:59:59-04:00"])
       #     ax.set_xlim(["00:00:00", "23:59:59"])
       xax = ax.get_xaxis()
       xax.set_major_locator(hours)
       xax.set_major_formatter(h_fmt)
       xax.set_minor_locator(mdates.HourLocator(interval = 1))
       xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
       xax.set_tick_params(which='major', pad=15)
    plt.savefig('subplots_sensordata/weatherdataplots/delta_bar/'+date+".png",bbox_inches="tight",dpi = 300)
    fig, axs = plt.subplots(9,figsize=(20, 10),sharex = False, sharey = False,gridspec_kw={'hspace': 0})

In [ ]:
def butter_lowpass_filter(data, cutoff, fs, order=5):
    # Filtering Helper functions
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = scisig.lfilter(b, a, data)
    return y

In [ ]:
condition_actual = ( (dpd['MIGRAINESTART'].notnull() ) ) & ( dpd['MIGRAINEEND'].notnull() )
condition_extra = ( (dpd['MIGRAINESTART'].isnull() ) ) & ( dpd['MIGRAINEEND'].notnull() )


In [ ]:
dpd.loc[condition_actual].count()
    

In [ ]:
#adding IBI and HRV
for f in range(0,len(files)):
#     io.StringIO(df.to_csv()), index_col=0
#     pd.read_csv(io.StringIO(f),index_col=0)
    df = pd.read_csv(files[f], index_col=0, parse_dates=[0])
    df_hrv = pd.read_csv(files_hrv[f], index_col=0, parse_dates=[0])
#     df4 = df3.drop_duplicates(subset='rownum', keep='last')
#     newdata_hrv = df_hrv.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')
    newdata_hrv = df_hrv.sort_index()
    newdata1 = df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')
    newdata1 = newdata1.sort_index()
    newdata10=newdata1.resample('10t').mean()
    newdata=newdata1.resample('1T').mean()
    data=newdata1[['AccelX','AccelY','AccelZ']]
    features, steps, motion = computeAllAccelerometerFeatures(data, time_frames)
    data["steps"] = steps
    data["motion"] = motion
    condition_dpd = ( (dpd['MIGRAINESTART_TimeZone'].notnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )

    date=str(newdata.index[0].date())
    df_wthr=df_weather.loc[date]
    print(date)
#     sfig_dpd01_2 = plt.subplot2grid((2,4),(1,0), colspan = 3, sharex = sfig_dpd01_1, sharey = sfig_dpd01_1)

    fig, axs = plt.subplots(9,figsize=(20, 10),sharex = True, sharey = False,gridspec_kw={'hspace': 0})
#     fig, axs1 = plt.subplots(1,figsize=(20, 7), sharex=True, sharey=False, gridspec_kw={'hspace': 0})
    fig.suptitle('Sensor Data Plots '+date) 
    objects = ('Sleep Duration', 'stress level', 'migraine index')
    y_pos = np.arange(len(objects))
    for i, row in dpd.loc[condition_actual, :].iterrows() : 
        for i in range(9):
            axs[i].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
            axs[i].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
            axs[i].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            axs[i].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
    for i, row in dpd.loc[condition_extra, :].iterrows() : 
        for i in range(9):
            axs[i].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

            axs[i].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
            axs[i].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
            axs[i].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
    
    axs[0].plot(newdata10.index, newdata10['Temp'],label='Body Temperature')
    #                axs[1].plot(peaks.index,peaks['peaks_plot'],'#4DBD33')
    axs[1].plot(newdata1.index,5*newdata1['peaks_plot'],'#4DBD33')
    axs[1].plot(newdata.index, newdata['filtered_eda'],label='Phasic')
    axs[1].plot(newdata.index,newdata['tonic'],label='Tonic')
    axs[2].plot(newdata.index, newdata['AccelX'],newdata.index, newdata['AccelY'],newdata.index, newdata['AccelZ'])
    axs[3].plot(data.index, (3*data['steps']),data.index, data['motion'])
    axs[4].plot(newdata_hrv.index, newdata_hrv['IBI'])
    axs[5].plot(newdata_hrv.index, newdata_hrv['HRV'])
    axs[8].plot(df_wthr.index, df_wthr['apparentTemperature'],label='Atmospheric Temperature')
    axs[6].plot(df_wthr.index, df_wthr['humidity'],label='Humidity')
    axs[7].plot(df_wthr.index, df_wthr['pressure'],label='Pressure')
    axs[6].plot(df_wthr.index, df_wthr['cloudCover'],label='Cloud Cover')
    delta_t=axs[8].twinx()
    delta_t.plot(df_wthr.index, df_wthr['deltaTemp_1'],'c-',label='Delta Temperature')
    delta_h_cc=axs[6].twinx()
    delta_h_cc.plot(df_wthr.index, df_wthr['deltahumidity_1'],'g-',label='Delta Humidity')
    
    delta_h_cc.plot(df_wthr.index, df_wthr['deltacloudCover_1'],'m-',label='Delta Cloud Cover')
    delta_p=axs[7].twinx()
    delta_p.plot(df_wthr.index, df_wthr['delta_pressure'],'r-',label='Delta Pressure')
    delta_h_cc.set_ylabel('delta H & CC',color='g',fontsize='x-small')
    delta_p.set_ylabel('delta Pressure',color='r',fontsize='x-small')
    delta_t.set_ylabel('delta Temp',color='c',fontsize='x-small')
    axs[0].legend(fontsize='xx-small')
    axs[1].legend(fontsize='xx-small')
    axs[2].legend(fontsize='xx-small')
    axs[3].legend(fontsize='xx-small')
    axs[4].legend(fontsize='xx-small')
    axs[5].legend(fontsize='xx-small')
    axs[6].legend(fontsize='xx-small')
    axs[7].legend(fontsize='xx-small')
    #            plt.xticks( rotation=25 )
    axs[0].set_ylabel('Temperature',fontsize='x-small')
#     axs[0].set_ylim(24 ,50)
    axs[1].set_ylabel('EDA',fontsize='x-small')
    data_min = min(newdata['EDA'])
    data_max =5
    axs[1].set_ylim(data_min ,data_max)
    axs[2].set_ylabel('Accelerometer',fontsize='x-small')
    axs[3].set_ylabel('Motion/Steps',fontsize='x-small')
    axs[4].set_ylabel('IBI',fontsize='x-small')
    axs[5].set_ylabel('HRV',fontsize='x-small')
    axs[8].set_ylabel('Temperature',fontsize='x-small')
    axs[6].set_ylabel('Hum & CC',fontsize='x-small')
    axs[7].set_ylabel('Pressure',fontsize='x-small')
#     axs[8].set_ylabel('CloudCover')
    axs[0].set_xlabel('Time',fontsize='x-small')

    # Hide x labels and tick labels for all but bottom plot.
    for ax in axs:
       ax.label_outer()
       hours = mdates.HourLocator(interval = 6)
       h_fmt = mdates.DateFormatter('%I:%M %p')
       ax.set_xlim([date+" 00:00:00-04:00", date+" 23:59:59-04:00"])
       #     ax.set_xlim(["00:00:00", "23:59:59"])
       xax = ax.get_xaxis()
       xax.set_major_locator(hours)
       xax.set_major_formatter(h_fmt)
       xax.set_minor_locator(mdates.HourLocator(interval = 1))
       xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
       xax.set_tick_params(which='major', pad=15)
    plt.savefig('subplots_sensordata/weatherdataplots/deltas/'+date+".png",bbox_inches="tight",dpi = 300)
#     plt.close()
#     axs1.savefig('subplots_sensordata/ '+date+"-da3.png",bbox_inches="tight",dpi = 900)
        

In [ ]:
#working code till accelerometer
for f in files:
#     io.StringIO(df.to_csv()), index_col=0
#     pd.read_csv(io.StringIO(f),index_col=0)
    df = pd.read_csv(f, index_col=0, parse_dates=[0])
#     df4 = df3.drop_duplicates(subset='rownum', keep='last')
    newdata1 = df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')
    newdata1 = newdata1.sort_index()
    newdata10=newdata1.resample('10t').mean()
    newdata=newdata1.resample('1T').mean()
    data=newdata1[['AccelX','AccelY','AccelZ']]
    features, steps, motion = computeAllAccelerometerFeatures(data, time_frames)
    data["steps"] = steps
    data["motion"] = motion
    
    
    condition_dpd = ( (dpd['MIGRAINESTART_TimeZone'].notnull() ) ) & ( dpd['MIGRAINEEND_TimeZone'].notnull() )
#     newdata = newdata.drop_duplicates(subset=, keep='first')
    date=str(newdata.index[0].date())
#     sfig_dpd01_2 = plt.subplot2grid((2,4),(1,0), colspan = 3, sharex = sfig_dpd01_1, sharey = sfig_dpd01_1)
#     data = data.append(df)
    fig, axs = plt.subplots(4,figsize=(20, 7))
#     fig, axs1 = plt.subplots(1,figsize=(20, 7), sharex=True, sharey=False, gridspec_kw={'hspace': 0})
    fig.suptitle('Sensor Data Plots'+date) 
    for i, row in dpd.loc[condition_dpd, :].iterrows() : 
    #print(t)
#         print(row['dt_diary'].date() + 'while date looping is '+date )
#         if(date==str(row['dt_diary'].date())):
        if (date in date_extra): 
            #extrapolated date time
#         print('extrapolated date time')
            for i in range(4):
                axs[i].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
                #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')
                axs[i].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'blue', alpha = 0.15)
                axs[i].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
                axs[i].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)
        else:
            print('real date time')
            for i in range(4):
                axs[i].axvline(row['dt_diary_tz'], color = 'green', linewidth = 2)
                #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

                axs[i].axvspan(row['MIGRAINESTART_TimeZone'],row['MIGRAINEEND_TimeZone'], color = 'orchid', alpha = 0.5)
                axs[i].axvline(row['MIGRAINESTART_TimeZone'], color = 'red', linewidth = 1)
                axs[i].axvline(row['MIGRAINEEND_TimeZone'], color = 'blue', linewidth = 1)

    axs[0].plot(newdata10.index, newdata10['Temp'])
    #                axs[1].plot(peaks.index,peaks['peaks_plot'],'#4DBD33')
    axs[1].plot(newdata1.index,newdata1['peaks_plot'],'#4DBD33')
    axs[1].plot(newdata.index, newdata['filtered_eda'])
    axs[1].plot(newdata.index,newdata['tonic'])
    axs[2].plot(newdata.index, newdata['AccelX'],newdata.index, newdata['AccelY'],newdata.index, newdata['AccelZ'])
    axs[3].plot(data.index, (3*data['steps']),data.index, data['motion'])

    plt.legend()
    #            plt.xticks( rotation=25 )
    
    axs[0].set_ylabel('Temperature')
    axs[1].set_ylabel('EDA')
    data_min = min(newdata['EDA'])
    data_max = max(newdata['EDA'])
    axs[1].set_ylim(data_min ,data_max)
    axs[2].set_ylabel('Accelerometer')
    axs[3].set_ylabel('Motion/Steps')
    axs[0].set_xlabel('Time')

    # Hide x labels and tick labels for all but bottom plot.
    for ax in axs:
       ax.label_outer()
    #        minutes = mdates.MinuteLocator(byminute=[0,15,30,45], interval = 1)
       hours = mdates.HourLocator(interval = 6)
       h_fmt = mdates.DateFormatter('%I:%M %p')
       ax.set_xlim([date+" 00:00:00-04:00", date+" 23:59:59-04:00"])
       #     ax.set_xlim(["00:00:00", "23:59:59"])
       xax = ax.get_xaxis()
       xax.set_major_locator(hours)
       xax.set_major_formatter(h_fmt)
       xax.set_minor_locator(mdates.HourLocator(interval = 1))
       xax.set_minor_formatter(mdates.DateFormatter('%I %p'))
       xax.set_tick_params(which='major', pad=15)
    plt.savefig('subplots_sensordata/ '+date+"-3.png",bbox_inches="tight",dpi = 300)
#     plt.close()
#     axs1.savefig('subplots_sensordata/ '+date+"-da3.png",bbox_inches="tight",dpi = 900)
        

In [ ]:
if True:

#     fig_dpd01 = plt.figure(figsize = (12,8))
    fig, ax = plt.subplots(figsize=(25,15))
#     fig_dpd01.set_label('')

#     legend_prop_dct = {'size' : 'medium'}        # Elements to pass to legend(prop = prop)

#     sfig_dpd01_1 = plt.subplot2grid((2,4),(0,0), colspan = 3)
#     plt.subplot2grid((2,4),(0,0), colspan = 3)
#     fig,sfig_dpd01_2 = plt.subplots(3,figsize=(20, 7), sharex=False, sharey=False, gridspec_kw={'hspace': 0})
    fig, axx = plt.subplots(4,figsize=(20, 7))
    condition_dpd = ( (dpd['MIGRAINESTART'].notnull() ) ) & ( dpd['MIGRAINEEND'].notnull() )
    condition = (dpd['MIGRAINESTART'].isnull() )  & ( dpd['MIGRAINEEND'].notnull())
    condition_ =  (dpd['HADMIGRAINE'] == True) & (dpd['MIGRAINESTART'].isnull() )  & ( dpd['MIGRAINEEND'].isnull())

    #Create legend handles
    mig0 = [mlines.Line2D([],[], color = 'lime', linewidth = 0, marker = 'o', markeredgewidth = 1, markeredgecolor = 'darkgreen', 
                          markersize = (1 * 15)**(1/2), label = 'Migraine Severity = 1'),
            mlines.Line2D([],[], color = 'lime', linewidth = 0, marker = 'o', markeredgewidth = 1, markeredgecolor = 'darkgreen', 
                          markersize = (2 * 15)**(1/2), label = 'Migraine Severity = 2')
           ]

    mig1 = mlines.Line2D([],[],color = 'green', linewidth = 2, linestyle = '-', label = "Migraine w/ Start and End Time")
    mig2 = mlines.Line2D([],[],color = 'navy', linewidth = 2, linestyle = '--', label ='Migraine w/ End Time Only')
    mig3 = mlines.Line2D([],[], color = 'orangered', linewidth = 2, linestyle = '--',label = "Migraine w/o Start or End Time")
    mig4 = mlines.Line2D([],[], color = 'black', linewidth = 1, linestyle = '-', alpha = 0.5, label = "No Migraine")

    # Create legend instance
#     axs[0].legend(handles = [mig1, mig2, mig3, mig4]+mig0, ncol = 1, numpoints = 1
#                        , loc = 'center left', title = 'Diary Entry Time Stamps:'
#                        , bbox_to_anchor=(1.01, .5),borderaxespad=0.
#                        , prop = legend_prop_dct)
    
    for i, row in dpd.loc[condition_dpd, :].iterrows() : 
        #print(t)
        print(row['dt_diary'].date())
        if(strdate==str(row['dt_diary'].date())):
            print('inside loop')
            axx[1].axvline(row['dt_diary'], color = 'green', linewidth = 2)
            #sfig_dpd01_2.axhline(row['dt_delta']/timedelta(hours = 1), linestyle = ':', color = 'black')

            axx[1].axvspan(row['MIGRAINESTART'],row['MIGRAINEEND'], color = 'orchid', alpha = 0.5)
            axx[1].axvline(row['MIGRAINESTART'], color = 'red', linewidth = 1)
            axx[1].axvline(row['MIGRAINEEND'], color = 'blue', linewidth = 1)
            #Add entries with partial entries
    hours = mdates.HourLocator(interval = 1)
    h_fmt = mdates.DateFormatter('%I:%M %p')
#     sfig_dpd01_2.set_xlim(xmin = dpd['dt_diary'].min() - timedelta(days = 0), xmax = dpd['dt_diary'].max() + timedelta(days = 0))
    ax.set_ylabel("Migraine Duration (hours)")
#     ax.set_xlim([strdate+" 00:00:00", strdate+" 23:59:59"])
    xax = ax.get_xaxis()
    xax.set_major_locator(hours)
    xax.set_major_formatter(h_fmt)
    # Create legend handles
    mig9 = mlines.Line2D([],[],linewidth = 2, color = 'green', linestyle = '-', label = 'Diary Entry of Migraine \nwith Start and End Times')
    mig8 = mlines.Line2D([],[], linewidth = 1, color = 'blue', linestyle = '-', label = 'Migraine End Time')
    mig7 = mlines.Line2D([],[], linewidth = 1, color = 'red', linestyle = '-', label = 'Migraine Start Time')
    mig6 = mpatches.Patch(color = 'orchid', alpha = 0.5, label = 'Migraine Duration/Span')
   
#     axs[0].legend(handles = [mig9, mig8, mig7, mig6] , ncol = 1, bbox_to_anchor=(1.01, 0.5), borderaxespad=0., loc = 'center left'
#                       , prop = legend_prop_dct)


In [ ]:
#fig_dpd01_.savefig(osp.join(dpd_dir,'fig2.png'), dpi = 1200)
dpd['SLEEPQUALITY'].unique()

In [ ]:
#del figE

In [ ]:
#del figT

In [ ]:
#df['time'].apply(pd.to_datetime)